# Lean 8 - Agents Autonomes pour Demonstration de Theoremes

**Navigation** : [← Lean-7-LLM-Integration](Lean-7-LLM-Integration.ipynb) | [Index](Lean-1-Setup.ipynb) | [Lean-9-LeanDojo →](Lean-9-LeanDojo.ipynb)

---


## Introduction

Ce notebook final de la serie explore la creation de **systemes multi-agents** capables de prouver des theoremes mathematiques de maniere **autonome**. Nous combinons les techniques des notebooks precedents avec les patterns d'orchestration agentique.

L'objectif est de construire un systeme qui peut :
1. Recevoir un enonce de theoreme
2. Rechercher des lemmes pertinents dans Mathlib
3. Generer des strategies de preuve
4. Verifier formellement avec Lean
5. Iterer jusqu'au succes

### Objectifs pedagogiques

1. Concevoir une architecture multi-agents pour theorem proving
2. Implementer des agents specialises (recherche, generation, verification)
3. Orchestrer la collaboration entre agents
4. Gerer les boucles de feedback et d'amelioration
5. Comprendre les techniques de Harmonic Aristotle et APOLLO

### Prerequis

- Notebooks **Lean-1** a **Lean-7** completes
- Notions de base sur les systemes multi-agents
- Cle API LLM (optionnel pour execution)

### Duree estimee : 55-60 minutes

---

## Architecture d'un Systeme Agentique pour Lean

### Vue d'ensemble

```
┌─────────────────────────────────────────────────────────────────────┐
│                     SYSTEME AGENTIQUE LEAN                          │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  ┌─────────────────┐                                               │
│  │   ORCHESTRATOR  │  <- Coordonne tous les agents                 │
│  │     Agent       │                                               │
│  └────────┬────────┘                                               │
│           │                                                        │
│  ┌────────┼────────┬────────────────┐                              │
│  │        │        │                │                              │
│  v        v        v                v                              │
│ ┌────┐  ┌────┐  ┌────┐         ┌────────┐                          │
│ │Search│ │Tactic│ │Proof│        │Memory  │                         │
│ │Agent│ │Agent│ │Verify│        │Store   │                         │
│ └──┬───┘ └──┬───┘ └──┬───┘        └────────┘                         │
│    │        │        │                                             │
│    v        v        v                                             │
│ ┌──────────────────────────────────────────────┐                   │
│ │               LEAN KERNEL                     │                   │
│ │  (Verification formelle + Mathlib)           │                   │
│ └──────────────────────────────────────────────┘                   │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

## 1. Agent de Recherche de Theoremes

### 1.1 Role

L'agent de recherche parcourt Mathlib pour trouver des lemmes pertinents au probleme.

In [100]:
from dataclasses import dataclass
from typing import List, Optional
import json
import re

@dataclass
class Lemma:
    """Represente un lemme Mathlib."""
    name: str
    statement: str
    namespace: str
    relevance_score: float = 0.0

class TheoremSearchAgent:
    """Agent de recherche de theoremes dans Mathlib."""

    # Base de lemmes connus (extensible)
    KNOWN_LEMMAS = [
        Lemma("Nat.add_zero", "n + 0 = n", "Nat"),
        Lemma("Nat.zero_add", "0 + n = n", "Nat"),
        Lemma("Nat.add_comm", "n + m = m + n", "Nat"),
        Lemma("Nat.add_assoc", "(n + m) + k = n + (m + k)", "Nat"),
        Lemma("Nat.mul_comm", "n * m = m * n", "Nat"),
        Lemma("Nat.mul_assoc", "(n * m) * k = n * (m * k)", "Nat"),
        Lemma("Nat.mul_zero", "n * 0 = 0", "Nat"),
        Lemma("Nat.zero_mul", "0 * n = 0", "Nat"),
        Lemma("Nat.mul_one", "n * 1 = n", "Nat"),
        Lemma("Nat.one_mul", "1 * n = n", "Nat"),
        Lemma("Nat.succ_add", "succ n + m = succ (n + m)", "Nat"),
        Lemma("Nat.add_succ", "n + succ m = succ (n + m)", "Nat"),
    ]

    def __init__(self, llm_client=None):
        self.llm = llm_client
        self.cache = {}  # Cache des recherches

    def search(self, goal: str, context: str = "") -> List[Lemma]:
        """
        Recherche des lemmes pertinents pour un but donne.

        Args:
            goal: Le but a prouver
            context: Contexte additionnel (hypotheses, etc.)

        Returns:
            Liste de lemmes tries par pertinence
        """
        # Verifier le cache
        cache_key = f"{goal}:{context}"
        if cache_key in self.cache:
            return self.cache[cache_key]

        # Analyser le but pour extraire les concepts
        concepts = self._extract_concepts(goal)

        # Rechercher dans la base de lemmes
        lemmas = self._search_mathlib(concepts, goal)

        # Scorer par pertinence
        scored = self._score_lemmas(lemmas, goal)

        # Mettre en cache
        self.cache[cache_key] = scored

        return scored

    def _extract_concepts(self, goal: str) -> List[str]:
        """Extrait les concepts mathematiques du but."""
        concepts = []
        goal_lower = goal.lower()

        # Mapping symboles -> concepts
        symbol_map = {
            "+": ["add"],
            "*": ["mul"],
            "0": ["zero"],
            "1": ["one"],
            "succ": ["succ"],
        }

        for symbol, keywords in symbol_map.items():
            if symbol in goal:
                concepts.extend(keywords)

        # Mots-cles explicites
        explicit_keywords = ["comm", "assoc", "zero", "one", "succ", "add", "mul"]
        for kw in explicit_keywords:
            if kw in goal_lower and kw not in concepts:
                concepts.append(kw)

        return list(set(concepts))

    def _search_mathlib(self, concepts: List[str], goal: str) -> List[Lemma]:
        """Recherche dans la base de lemmes connus."""
        if not concepts:
            # Fallback: retourner quelques lemmes de base
            return self.KNOWN_LEMMAS[:4]

        # Filtrer par concepts
        matches = []
        for lemma in self.KNOWN_LEMMAS:
            name_lower = lemma.name.lower()
            if any(c in name_lower for c in concepts):
                matches.append(Lemma(lemma.name, lemma.statement, lemma.namespace, 0.0))

        return matches if matches else self.KNOWN_LEMMAS[:3]

    def _score_lemmas(self, lemmas: List[Lemma], goal: str) -> List[Lemma]:
        """Score les lemmes par pertinence."""
        # Normaliser le but
        goal_normalized = goal.replace(" ", "").lower()

        for lemma in lemmas:
            # Score base sur la correspondance structurelle
            stmt_normalized = lemma.statement.replace(" ", "").lower()

            # Score exact match
            if goal_normalized == stmt_normalized:
                lemma.relevance_score = 1.0
            # Score partial match
            elif goal_normalized in stmt_normalized or stmt_normalized in goal_normalized:
                lemma.relevance_score = 0.8
            else:
                # Score par tokens communs
                goal_tokens = set(re.findall(r'[a-z]+|[0-9]+|[+*=]', goal_normalized))
                stmt_tokens = set(re.findall(r'[a-z]+|[0-9]+|[+*=]', stmt_normalized))
                common = goal_tokens & stmt_tokens
                lemma.relevance_score = len(common) / max(len(goal_tokens), 1) * 0.6

        return sorted(lemmas, key=lambda l: l.relevance_score, reverse=True)

# Test
search_agent = TheoremSearchAgent()
results = search_agent.search("n + 0 = n")
print("Lemmes trouves:")
for lemma in results:
    print(f"  {lemma.name}: {lemma.statement} (score: {lemma.relevance_score:.2f})")


Lemmes trouves:
  Nat.add_zero: n + 0 = n (score: 1.00)
  Nat.zero_add: 0 + n = n (score: 0.60)
  Nat.add_comm: n + m = m + n (score: 0.45)
  Nat.add_assoc: (n + m) + k = n + (m + k) (score: 0.45)
  Nat.mul_zero: n * 0 = 0 (score: 0.45)
  Nat.zero_mul: 0 * n = 0 (score: 0.45)
  Nat.succ_add: succ n + m = succ (n + m) (score: 0.45)
  Nat.add_succ: n + succ m = succ (n + m) (score: 0.45)


## 2. Agent de Generation de Tactiques

### 2.1 Role

L'agent de tactiques genere des sequences de tactiques Lean pour prouver le but.

In [101]:
from enum import Enum
from typing import Tuple

class TacticType(Enum):
    DIRECT = "direct"       # exact, rfl
    REWRITE = "rewrite"     # rw, simp
    SPLIT = "split"         # constructor, cases
    INDUCTION = "induction" # induction, recursion
    AUTO = "auto"           # omega, ring, linarith

@dataclass
class TacticSuggestion:
    """Une suggestion de tactique avec son contexte."""
    tactic: str
    tactic_type: TacticType
    confidence: float
    explanation: str

class TacticGeneratorAgent:
    """Agent de generation de tactiques."""
    
    def __init__(self, llm_client=None):
        self.llm = llm_client
        self.history = []  # Historique des tentatives
    
    def generate(self, goal: str, context: List[str], 
                 available_lemmas: List[Lemma]) -> List[TacticSuggestion]:
        """
        Genere des tactiques pour un but donne.
        
        Args:
            goal: Le but courant
            context: Les hypotheses disponibles
            available_lemmas: Lemmes suggeres par l'agent de recherche
        
        Returns:
            Liste de suggestions de tactiques
        """
        suggestions = []
        
        # Strategie 1: Tactiques directes
        if "=" in goal:
            suggestions.append(TacticSuggestion(
                "rfl", TacticType.DIRECT, 0.9,
                "Reflexivite - verifie si les deux cotes sont identiques"
            ))
        
        # Strategie 2: Utiliser les lemmes disponibles
        for lemma in available_lemmas[:3]:
            suggestions.append(TacticSuggestion(
                f"exact {lemma.name}", TacticType.DIRECT, 
                lemma.relevance_score,
                f"Appliquer {lemma.name}: {lemma.statement}"
            ))
            suggestions.append(TacticSuggestion(
                f"rw [{lemma.name}]", TacticType.REWRITE,
                lemma.relevance_score * 0.8,
                f"Reecrire avec {lemma.name}"
            ))
        
        # Strategie 3: Tactiques automatiques
        if any(op in goal for op in ["+", "-", "<", ">", "<=", ">="]):
            suggestions.append(TacticSuggestion(
                "omega", TacticType.AUTO, 0.7,
                "Arithmetique de Presburger automatique"
            ))
        
        if "*" in goal or "^" in goal:
            suggestions.append(TacticSuggestion(
                "ring", TacticType.AUTO, 0.7,
                "Algebre polynomiale automatique"
            ))
        
        # Strategie 4: Simp comme fallback
        suggestions.append(TacticSuggestion(
            "simp", TacticType.REWRITE, 0.5,
            "Simplification automatique"
        ))
        
        # Trier par confiance
        return sorted(suggestions, key=lambda s: s.confidence, reverse=True)
    
    def generate_sequence(self, goal: str, context: List[str],
                          available_lemmas: List[Lemma],
                          max_depth: int = 5) -> List[str]:
        """
        Genere une sequence complete de tactiques.
        """
        sequence = []
        current_goal = goal
        
        for _ in range(max_depth):
            suggestions = self.generate(current_goal, context, available_lemmas)
            if not suggestions:
                break
            
            best = suggestions[0]
            sequence.append(best.tactic)
            
            # Simuler la progression (dans la realite, Lean nous dirait le nouveau but)
            if best.tactic_type == TacticType.DIRECT:
                break  # Preuve complete
        
        return sequence

# Test
tactic_agent = TacticGeneratorAgent()
lemmas = search_agent.search("n + 0 = n")
suggestions = tactic_agent.generate("n + 0 = n", [], lemmas)

print("Tactiques suggerees:")
for s in suggestions[:5]:
    print(f"  [{s.confidence:.2f}] {s.tactic} - {s.explanation}")

Tactiques suggerees:
  [1.00] exact Nat.add_zero - Appliquer Nat.add_zero: n + 0 = n
  [0.90] rfl - Reflexivite - verifie si les deux cotes sont identiques
  [0.80] rw [Nat.add_zero] - Reecrire avec Nat.add_zero
  [0.70] omega - Arithmetique de Presburger automatique
  [0.60] exact Nat.zero_add - Appliquer Nat.zero_add: 0 + n = n


## 3. Agent de Verification

### 3.1 Role

L'agent de verification execute le code Lean et analyse les resultats.

In [102]:
@dataclass
class VerificationResult:
    """Resultat de la verification Lean."""
    success: bool
    error_message: Optional[str] = None
    remaining_goals: List[str] = None
    execution_time: float = 0.0

class ProofVerifierAgent:
    """Agent de verification des preuves."""
    
    def __init__(self, lean_path: str = "lean"):
        self.lean_path = lean_path
        self.verified_count = 0
        self.failed_count = 0
    
    def verify(self, theorem: str, proof: str) -> VerificationResult:
        """
        Verifie une preuve avec Lean.
        
        Args:
            theorem: L'enonce du theoreme
            proof: La preuve proposee (sequence de tactiques)
        
        Returns:
            Resultat de la verification
        """
        # Construire le code Lean complet
        lean_code = self._build_lean_code(theorem, proof)
        
        # Simuler l'execution Lean
        # (Dans un vrai systeme, on utiliserait subprocess ou lean-dojo)
        result = self._simulate_lean_execution(lean_code)
        
        # Mettre a jour les statistiques
        if result.success:
            self.verified_count += 1
        else:
            self.failed_count += 1
        
        return result
    
    def _build_lean_code(self, theorem: str, proof: str) -> str:
        """Construit le code Lean complet."""
        return f"""
{theorem} := by
  {proof}
        """.strip()
    
    def _simulate_lean_execution(self, code: str) -> VerificationResult:
        """
        Simule l'execution Lean.
        Dans un vrai systeme, utiliser lean-dojo ou subprocess.
        """
        # Heuristiques simples pour la simulation
        if "rfl" in code or "exact Nat.add_zero" in code:
            return VerificationResult(success=True)
        elif "sorry" in code:
            return VerificationResult(
                success=False,
                error_message="declaration uses 'sorry'"
            )
        else:
            # Simuler une reussite aleatoire
            import random
            if random.random() > 0.3:
                return VerificationResult(success=True)
            else:
                return VerificationResult(
                    success=False,
                    error_message="tactic failed"
                )
    
    def get_stats(self) -> dict:
        """Retourne les statistiques de verification."""
        total = self.verified_count + self.failed_count
        return {
            "verified": self.verified_count,
            "failed": self.failed_count,
            "success_rate": self.verified_count / max(total, 1)
        }

# Test
verifier = ProofVerifierAgent()
result = verifier.verify(
    "theorem test (n : Nat) : n + 0 = n",
    "exact Nat.add_zero n"
)
print(f"Verification: {'Succes' if result.success else 'Echec'}")
if result.error_message:
    print(f"Erreur: {result.error_message}")

Verification: Succes


## 4. Agent Orchestrateur

### 4.1 Role

L'orchestrateur coordonne tous les agents pour resoudre un probleme.

In [103]:
@dataclass
class ProofAttempt:
    """Enregistre une tentative de preuve."""
    theorem: str
    tactics: List[str]
    result: VerificationResult
    iteration: int

class OrchestratorAgent:
    """
    Agent orchestrateur qui coordonne le systeme multi-agents.
    """
    
    def __init__(self):
        self.search_agent = TheoremSearchAgent()
        self.tactic_agent = TacticGeneratorAgent()
        self.verifier = ProofVerifierAgent()
        self.history: List[ProofAttempt] = []
        self.max_iterations = 10
    
    def prove(self, theorem: str) -> Tuple[bool, Optional[str]]:
        """
        Tente de prouver un theoreme.
        
        Args:
            theorem: L'enonce du theoreme
        
        Returns:
            (succes, preuve) ou (echec, None)
        """
        print(f"\n{'='*60}")
        print(f"Debut de la preuve: {theorem}")
        print(f"{'='*60}\n")
        
        for iteration in range(self.max_iterations):
            print(f"--- Iteration {iteration + 1} ---")
            
            # Etape 1: Rechercher des lemmes pertinents
            goal = self._extract_goal(theorem)
            lemmas = self.search_agent.search(goal)
            print(f"Lemmes trouves: {[l.name for l in lemmas[:3]]}")
            
            # Etape 2: Generer des tactiques
            tactics = self.tactic_agent.generate_sequence(
                goal, [], lemmas
            )
            proof = "\n  ".join(tactics)
            print(f"Tactiques generees: {tactics}")
            
            # Etape 3: Verifier
            result = self.verifier.verify(theorem, proof)
            
            # Enregistrer la tentative
            self.history.append(ProofAttempt(
                theorem, tactics, result, iteration
            ))
            
            if result.success:
                print(f"\nPreuve trouvee!")
                return True, proof
            else:
                print(f"Echec: {result.error_message}")
                # Apprendre de l'echec pour la prochaine iteration
                self._learn_from_failure(result)
        
        print(f"\nEchec apres {self.max_iterations} iterations")
        return False, None
    
    def _extract_goal(self, theorem: str) -> str:
        """Extrait le but du theoreme."""
        # Simplification: prendre la partie apres le ":"
        if ":" in theorem:
            return theorem.split(":", 1)[1].strip()
        return theorem
    
    def _learn_from_failure(self, result: VerificationResult):
        """Ajuste la strategie basee sur l'echec."""
        # Dans un vrai systeme, on ajusterait les poids,
        # eviterait les tactiques qui echouent, etc.
        pass
    
    def get_statistics(self) -> dict:
        """Retourne les statistiques du systeme."""
        return {
            "total_attempts": len(self.history),
            "verifier_stats": self.verifier.get_stats()
        }

# Demonstration
orchestrator = OrchestratorAgent()
success, proof = orchestrator.prove(
    "theorem add_zero (n : Nat) : n + 0 = n"
)

if success:
    print(f"\nPreuve finale:\n{proof}")


Debut de la preuve: theorem add_zero (n : Nat) : n + 0 = n

--- Iteration 1 ---
Lemmes trouves: ['Nat.add_zero', 'Nat.zero_add', 'Nat.add_comm']
Tactiques generees: ['rfl']

Preuve trouvee!

Preuve finale:
rfl


## 🎯 Architecture du Système Multi-Agents

### Vue d'ensemble

Notre système utilise **5 agents spécialisés** qui collaborent pour prouver des théorèmes Lean :

1. **SearchAgent** : Recherche de lemmes pertinents dans Mathlib
2. **TacticAgent** : Génération de tactiques Lean appropriées
3. **VerifierAgent** : Vérification formelle des preuves
4. **CriticAgent** : Analyse et suggestions d'amélioration
5. **CoordinatorAgent** : Orchestration et décisions stratégiques

### Pourquoi 5 agents ?

Chaque agent a une **responsabilité unique** (principe de séparation des préoccupations) :

- **Séparation des compétences** : Recherche ≠ Génération ≠ Vérification
- **Spécialisation** : Chaque LLM est prompté pour une tâche précise
- **Robustesse** : Si un agent échoue, les autres continuent
- **Traçabilité** : On sait quel agent a pris quelle décision

### Communication : État partagé vs Message passing

Deux approches classiques en multi-agents :

| **Message Passing** | **État Partagé** (notre choix) |
|---------------------|--------------------------------|
| Agents s'envoient des messages | Tous les agents lisent/écrivent un état central |
| Décentralisé | Centralisé |
| Complexe à orchestrer | Facile à suivre |
| Pas de snapshot global | Snapshot complet à chaque itération |

**Pourquoi état partagé ?**

- Besoin de **cohérence globale** (historique des tactiques, métriques)
- **Debugging facilité** : On peut inspecter l'état après chaque tour
- **Snapshots JSON** : Permet de reproduire exactement une session
- Semantic Kernel supporte ce pattern avec les **plugins**

## 5. Integration avec Semantic Kernel (Python)

### 5.1 Vue d'ensemble

Microsoft **Semantic Kernel** est un SDK qui permet d'orchestrer des LLMs avec des plugins, de la memoire et des agents intelligents. Nous allons implementer un systeme multi-agents pour theorem proving inspire des patterns utilises dans l'analyse argumentative (voir `Argument_Analysis` notebooks).

**Composants cles** :
- **Kernel** : Point d'entree principal, configure les services LLM
- **Plugins** : Fonctions appelables par les agents (decorated avec `@kernel_function`)
- **Agents** : Entites autonomes avec instructions et capacites
- **Orchestration** : Strategies de selection et terminaison des agents

### 5.2 Dependances

```python
# Installation
pip install semantic-kernel openai python-dotenv
```

## 📊 État Partagé : La Classe `ProofState`

La classe `ProofState` est le **cœur du système**. Elle contient :

### 1. Phase de preuve (`ProofPhase` enum)
```
INIT → SEARCH → TACTIC_GEN → VERIFICATION → REFINEMENT → COMPLETE
```

Chaque phase détermine **quel agent agit** :
- `INIT` → CoordinatorAgent décide de la stratégie
- `SEARCH` → SearchAgent cherche des lemmes
- `TACTIC_GEN` → TacticAgent génère une tactique
- `VERIFICATION` → VerifierAgent teste la preuve
- `REFINEMENT` → CriticAgent analyse et ajuste
- `COMPLETE` → Session terminée

### 2. Stratégie de preuve (`ProofStrategy` enum)

```python
EXPLORATION   # Recherche large de lemmes
REFINEMENT    # Ajustement d'une preuve existante
VALIDATION    # Vérification formelle
RECOVERY      # Récupération après erreur
```

La stratégie influence **quels lemmes rechercher** et **quelles tactiques essayer**.

### 3. Historique et métriques

- `tactic_history` : Liste de toutes les tactiques essayées (succès + échecs)
- `verification_results` : Résultats des vérifications Lean
- `current_proof` : Preuve en construction
- `error_count` : Nombre d'erreurs rencontrées

### 4. Snapshots JSON

À chaque itération, on peut sauvegarder l'état complet en JSON :

```json
{
  "phase": "TACTIC_GEN",
  "strategy": "EXPLORATION",
  "iteration": 5,
  "current_goal": "n + 0 = n",
  "tactic_history": [...],
  "current_proof": ["intro n", "rw [Nat.add_zero]"]
}
```

**Utilité** : Debugging, reproduction de bugs, benchmarking.

In [104]:
# =============================================================================
# Section 8.1 - ProofState: Etat Partage pour Multi-Agents
# =============================================================================
# Pattern inspire de RhetoricalAnalysisState dans Argument_Analysis
# Permet la synchronisation entre agents avec designation explicite

import os
import sys
import json
import time
import uuid
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Any, Tuple
from datetime import datetime
from enum import Enum
from pathlib import Path

# --- Detection robuste du repertoire du notebook ---
# Fonctionne sous Windows, Linux, et WSL
notebook_dir = None

# Chemins connus (Windows et WSL)
KNOWN_PATHS = [
    Path("/mnt/d/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean"),  # WSL
    Path("/mnt/c/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean"),  # WSL (C:)
    Path("d:/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean"),      # Windows
    Path("D:/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean"),      # Windows
]

# Strategie 1: Variable d'environnement LEAN_NOTEBOOK_DIR
if os.getenv("LEAN_NOTEBOOK_DIR"):
    notebook_dir = Path(os.getenv("LEAN_NOTEBOOK_DIR"))
    if not (notebook_dir / "lean_runner.py").exists():
        notebook_dir = None

# Strategie 2: Chemins connus
if not notebook_dir:
    for known_path in KNOWN_PATHS:
        if known_path.exists() and (known_path / "lean_runner.py").exists():
            notebook_dir = known_path
            break

# Strategie 3: Chercher dans cwd et parents
if not notebook_dir:
    cwd = Path.cwd()
    candidates = [cwd, cwd / "MyIA.AI.Notebooks" / "SymbolicAI" / "Lean"]

    # Remonter jusqu'a 5 niveaux
    current = cwd
    for _ in range(5):
        candidates.append(current)
        lean_path = current / "MyIA.AI.Notebooks" / "SymbolicAI" / "Lean"
        if lean_path.exists():
            candidates.append(lean_path)
        if current.parent == current:
            break
        current = current.parent

    for candidate in candidates:
        if candidate.exists() and (candidate / "lean_runner.py").exists():
            notebook_dir = candidate
            break

# Strategie 4: Fallback sur cwd
if not notebook_dir:
    notebook_dir = Path.cwd()
    print(f"[WARN] lean_runner.py non trouve, fallback sur: {notebook_dir}")

# --- Charger .env ---
try:
    from dotenv import load_dotenv
    env_paths = [
        notebook_dir / ".env",
        notebook_dir.parent / ".env",
        Path.home() / ".env"
    ]
    for p in env_paths:
        if p.exists():
            load_dotenv(p, override=True)
            print(f"Configuration chargee depuis: {p}")
            break
    else:
        print("Aucun fichier .env trouve")
except ImportError:
    print("python-dotenv non installe")

# --- Importer lean_runner.py ---
if notebook_dir and str(notebook_dir) not in sys.path:
    sys.path.insert(0, str(notebook_dir))

try:
    from lean_runner import LeanRunner, LeanResult
    print(f"lean_runner importe avec succes depuis {notebook_dir}")
except ImportError as e:
    print(f"ERREUR: Impossible d'importer lean_runner: {e}")
    print(f"Repertoire de travail: {Path.cwd()}")
    print(f"notebook_dir detecte: {notebook_dir}")
    print(f"sys.path: {sys.path[:5]}")
    raise

# --- Enumerations ---

class ProofStrategy(Enum):
    """Strategie de preuve en cours."""
    EXPLORATION = "exploration"      # Recherche initiale de lemmes
    REFINEMENT = "refinement"        # Affinage des tactiques
    VALIDATION = "validation"        # Verification finale
    RECOVERY = "recovery"            # Recuperation apres echecs

class TacticDifficulty(Enum):
    """Niveau de difficulte des tactiques."""
    SIMPLE = "simple"      # rfl, exact, omega
    INTERMEDIATE = "intermediate"  # simp, ring, linarith
    ADVANCED = "advanced"  # induction, cases

class ProofPhase(Enum):
    """Phase de la boucle de preuve."""
    INIT = "init"
    SEARCH = "search"
    GENERATE = "generate"
    VERIFY = "verify"
    ANALYZE = "analyze"
    COMPLETE = "complete"
    FAILED = "failed"

# --- ProofState: Etat partage entre agents ---

@dataclass
class TacticAttempt:
    """Une tentative de tactique."""
    tactic: str
    success: bool
    error: Optional[str] = None
    timestamp: datetime = field(default_factory=datetime.now)
    state_before: Optional[str] = None
    confidence: Optional[float] = None
    explanation: Optional[str] = None

@dataclass
class ProofState:
    """
    Etat partage entre les agents pour la preuve d'un theoreme.
    Permet la coordination sans couplage fort.
    """
    # Identifiants
    session_id: str = field(default_factory=lambda: str(uuid.uuid4())[:8])
    theorem_name: str = ""
    theorem_statement: str = ""

    # Etat de la preuve
    current_goal: str = ""
    current_proof: List[str] = field(default_factory=list)
    phase: ProofPhase = ProofPhase.INIT
    strategy: ProofStrategy = ProofStrategy.EXPLORATION

    # Resultats des agents
    discovered_lemmas: List[str] = field(default_factory=list)
    generated_tactics: List[str] = field(default_factory=list)
    tactic_history: List[TacticAttempt] = field(default_factory=list)

    # Metriques
    iteration: int = 0
    max_iterations: int = 10
    start_time: datetime = field(default_factory=datetime.now)

    # Erreurs et diagnostics
    last_error: Optional[str] = None
    final_proof: Optional[str] = None
    error_count: int = 0

    # Verification tracking
    verification_results: List[Dict[str, Any]] = field(default_factory=list)
    total_lean_time_ms: float = 0.0

    # Agent designation for orchestration
    _next_agent: Optional[str] = field(default=None, repr=False)

    def add_tactic_attempt(self, tactic: str, state_before: Optional[str] = None,
                           confidence: Optional[float] = None, explanation: Optional[str] = None,
                           success: bool = False, error: Optional[str] = None) -> str:
        """Enregistre une tentative de tactique."""
        attempt_id = f"attempt_{len(self.tactic_history) + 1}"
        self.tactic_history.append(TacticAttempt(
            tactic=tactic,
            success=success,
            error=error,
            state_before=state_before,
            confidence=confidence,
            explanation=explanation
        ))
        if success:
            self.current_proof.append(tactic)
        else:
            self.error_count += 1
            self.last_error = error
        return attempt_id

    def add_lemma(self, name: str, statement: str, namespace: str = "", relevance: float = 0.5) -> str:
        """Ajoute un lemme decouvert a la liste."""
        lemma_id = f"{namespace}.{name}" if namespace else name
        lemma_info = f"{lemma_id}: {statement} (relevance: {relevance})"
        if lemma_info not in self.discovered_lemmas:
            self.discovered_lemmas.append(lemma_info)
        return lemma_id

    def get_context_summary(self) -> str:
        """Resume le contexte pour les agents."""
        return f"""
Theoreme: {self.theorem_name}
Enonce: {self.theorem_statement}
But actuel: {self.current_goal}
Phase: {self.phase.value}
Strategie: {self.strategy.value}
Iteration: {self.iteration}/{self.max_iterations}
Tactiques reussies: {len(self.current_proof)}
Erreurs: {self.error_count}
Derniere erreur: {self.last_error or 'Aucune'}
""".strip()


    

    # --- Properties for compatibility ---
    @property
    def tactics_history(self) -> List[TacticAttempt]:
        """Alias pour tactic_history (compatibilite)."""
        return self.tactic_history

    @property
    def proof_complete(self) -> bool:
        """True si la preuve est complete."""
        return self.phase == ProofPhase.COMPLETE
    
    @proof_complete.setter
    def proof_complete(self, value: bool):
        """Definit la completion de la preuve."""
        if value:
            self.phase = ProofPhase.COMPLETE
        elif self.phase == ProofPhase.COMPLETE:
            self.phase = ProofPhase.VERIFY
    
    @property
    def iteration_count(self) -> int:
        """Alias pour iteration (compatibilite)."""
        return self.iteration
    
    @iteration_count.setter
    def iteration_count(self, value: int):
        """Definit le compteur d'iterations."""
        self.iteration = value

    def increment_iteration(self):
        """Incremente le compteur d'iterations."""
        self.iteration += 1
    
    def designate_next_agent(self, agent_name: str):
        """Designe l'agent qui doit intervenir ensuite."""
        self._next_agent = agent_name
    
    def consume_next_agent_designation(self) -> Optional[str]:
        """Retourne et efface la designation d'agent."""
        agent = self._next_agent
        self._next_agent = None
        return agent
    
    def get_state_snapshot(self, summarize: bool = True) -> Dict[str, Any]:
        """Retourne un snapshot de l'etat pour les plugins."""
        if summarize:
            return {
                "session_id": self.session_id,
                "theorem": self.theorem_statement,
                "goal": self.current_goal,
                "phase": self.phase.value,
                "strategy": self.strategy.value,
                "iteration": f"{self.iteration}/{self.max_iterations}",
                "proof_steps": len(self.current_proof),
                "discovered_lemmas": len(self.discovered_lemmas),
                "errors": self.error_count,
                "last_error": self.last_error
            }
        else:
            return self.to_dict()


    def add_verification(self, attempt_id: str, success: bool, output: str, errors: str,
                         remaining_goals: Optional[str] = None, exec_time_ms: float = 0.0,
                         mode: str = "subprocess") -> str:
        """Enregistre un résultat de vérification Lean."""
        verif_id = f"verif_{len(self.verification_results) + 1}"
        self.verification_results.append({
            "id": verif_id,
            "attempt_id": attempt_id,
            "success": success,
            "output": output,
            "errors": errors,
            "remaining_goals": remaining_goals,
            "exec_time_ms": exec_time_ms,
            "mode": mode,
            "timestamp": datetime.now().isoformat()
        })
        return verif_id


    def set_proof_complete(self, proof: str):
        """Marque la preuve comme terminée et change la phase."""
        self.final_proof = proof
        self.phase = ProofPhase.COMPLETE


    def set_strategy(self, strategy: 'ProofStrategy'):
        """Change la stratégie de preuve."""
        self.strategy = strategy

    def to_dict(self) -> Dict[str, Any]:
        """Serialise l'etat."""
        return {
            "session_id": self.session_id,
            "theorem_name": self.theorem_name,
            "theorem_statement": self.theorem_statement,
            "current_goal": self.current_goal,
            "current_proof": self.current_proof,
            "phase": self.phase.value,
            "strategy": self.strategy.value,
            "discovered_lemmas": self.discovered_lemmas,
            "generated_tactics": self.generated_tactics,
            "iteration": self.iteration,
            "max_iterations": self.max_iterations,
            "error_count": self.error_count,
            "last_error": self.last_error
        }

# --- Test de l'initialisation ---
print("\n" + "="*60)
print("ProofState initialise avec succes")
print(f"LeanRunner disponible: {LeanRunner is not None}")
print("="*60)


Configuration chargee depuis: /mnt/d/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean/.env
lean_runner importe avec succes depuis /mnt/d/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean

ProofState initialise avec succes
LeanRunner disponible: True


### 8.2-8.5 Plugins Semantic Kernel

L'architecture utilise 4 plugins specialises, chacun exposant des fonctions via `@kernel_function`:

| Plugin | Role | Fonctions cles |
|--------|------|----------------|
| **ProofStateManagerPlugin** | Gestion de l'etat | get_proof_state, add_lemma, designate_next_agent |
| **LeanSearchPlugin** | Recherche Mathlib | search_mathlib_lemmas, check_lemma_type |
| **LeanTacticPlugin** | Generation tactiques | generate_tactics, analyze_tactic_failure |
| **LeanVerificationPlugin** | Verification Lean | verify_proof, verify_tactic_step |

Ce pattern permet aux agents d'appeler ces fonctions automatiquement grace au `FunctionChoiceBehavior.Auto()` de Semantic Kernel.

## 🔌 Plugins Semantic Kernel : Exposer l'État aux Agents

### Problème

Les agents LLM ne peuvent pas accéder directement à `ProofState` (objet Python).

### Solution : Plugins

Un **plugin Semantic Kernel** expose des méthodes Python comme **fonctions appelables par le LLM**.

```python
@kernel_function(
    description="Enregistre une tentative de tactique",
    name="log_tactic_attempt"
)
def log_tactic_attempt(self, tactic: str, confidence: float) -> str:
    attempt_id = self._state.add_tactic_attempt(tactic, confidence=confidence)
    return f"Tactique {tactic} enregistrée avec ID {attempt_id}"
```

### Décorateur `@kernel_function`

- `description` : Ce que le LLM voit ("À quoi sert cette fonction ?")
- `name` : Nom de la fonction pour le LLM
- Paramètres : Doivent correspondre **EXACTEMENT** à ce que le plugin appelle

### Les 4 plugins

1. **log_tactic_attempt** : Enregistrer une tactique essayée
2. **add_verification_result** : Enregistrer le résultat Lean
3. **set_proof_strategy** : Changer la stratégie de recherche
4. **mark_proof_complete** : Déclarer la preuve terminée

### Pourquoi c'est critique ?

Sans plugins, le LLM ne peut que **parler** de preuves. Avec plugins, il peut **agir** :

- Essayer des tactiques
- Vérifier formellement
- Ajuster sa stratégie en temps réel

In [ ]:
# =============================================================================
# Section 8.2-8.5 - Plugins Semantic Kernel
# =============================================================================
# Architecture en 4 plugins specialises:
# - ProofStateManagerPlugin: Gestion de l'etat partage
# - LeanSearchPlugin: Recherche de lemmes Mathlib
# - LeanTacticPlugin: Generation de tactiques
# - LeanVerificationPlugin: Verification avec lean_runner.py

# Import du decorateur kernel_function
try:
    from semantic_kernel.functions import kernel_function
    SK_AVAILABLE = True
    print("Semantic Kernel disponible - utilisation des vrais decorateurs")
except ImportError:
    SK_AVAILABLE = False
    print("Semantic Kernel non disponible - mode simulation")
    # Decorateur de simulation
    def kernel_function(description="", name=None):
        def decorator(func):
            func._sk_function = True
            func._sk_description = description
            func._sk_name = name or func.__name__
            return func
        return decorator

# =============================================================================
# 8.2 ProofStateManagerPlugin
# =============================================================================

class ProofStateManagerPlugin:
    """
    Plugin pour gerer l'etat partage de la preuve.
    Expose les methodes de ProofState via @kernel_function.
    """

    def __init__(self, state: ProofState):
        self._state = state

    @kernel_function(
        description="Obtient un apercu de l'etat actuel de la preuve (theoreme, lemmes, tactiques, etc.)",
        name="get_proof_state"
    )
    def get_proof_state(self, summarize: bool = True) -> str:
        """Retourne l'etat actuel sous forme JSON."""
        snapshot = self._state.get_state_snapshot(summarize=summarize)
        return json.dumps(snapshot, indent=2, ensure_ascii=False)

    @kernel_function(
        description="Ajoute un lemme decouvert a l'etat partage",
        name="add_discovered_lemma"
    )
    def add_discovered_lemma(
        self, name: str, statement: str, namespace: str = "", relevance: float = 0.5
    ) -> str:
        """Enregistre un lemme trouve par SearchAgent."""
        lemma_id = self._state.add_lemma(name, statement, namespace, relevance)
        return f"Lemme ajoute: {lemma_id} ({name})"

    @kernel_function(
        description="Enregistre une tentative de tactique avec son niveau de confiance",
        name="log_tactic_attempt"
    )
    def log_tactic_attempt(
        self, tactic: str, state_before: str, confidence: float = 0.5, explanation: str = ""
    ) -> str:
        """Enregistre une tactique tentee par TacticAgent."""
        attempt_id = self._state.add_tactic_attempt(tactic, state_before, confidence, explanation)
        return f"Tactique enregistree: {attempt_id}"

    @kernel_function(
        description="Enregistre le resultat d'une verification Lean",
        name="add_verification_result"
    )
    def add_verification_result(
        self, attempt_id: str, success: bool, output: str, errors: str,
        remaining_goals: str = "", exec_time_ms: float = 0.0
    ) -> str:
        """Enregistre un resultat de verification."""
        verif_id = self._state.add_verification(
            attempt_id, success, output, errors,
            remaining_goals if remaining_goals else None, exec_time_ms, "subprocess"
        )
        status = "OK" if success else "ECHEC"
        return f"Verification {verif_id}: {status}"

    @kernel_function(
        description="Designe l'agent qui doit parler au prochain tour. IMPORTANT: utiliser le nom exact.",
        name="designate_next_agent"
    )
    def designate_next_agent(self, agent_name: str) -> str:
        """Delegue au prochain agent."""
        valid_agents = ["SearchAgent", "TacticAgent", "VerifierAgent", "CriticAgent", "CoordinatorAgent"]
        if agent_name not in valid_agents:
            return f"ERREUR: Agent invalide '{agent_name}'. Valides: {valid_agents}"
        self._state.designate_next_agent(agent_name)
        return f"Prochain agent: {agent_name}"

    @kernel_function(
        description="Marque la preuve comme terminee avec le code final",
        name="set_proof_complete"
    )
    def set_proof_complete(self, proof_code: str) -> str:
        """Marque la preuve comme reussie."""
        self._state.set_proof_complete(proof_code)
        return f"PREUVE COMPLETE! Code: {proof_code[:100]}..."

    @kernel_function(
        description="Change la strategie de preuve (exploration, refinement, validation, recovery)",
        name="set_proof_strategy"
    )
    def set_proof_strategy(self, strategy: str) -> str:
        """Change la strategie de preuve."""
        try:
            self._state.set_strategy(ProofStrategy(strategy))
            return f"Strategie changee: {strategy}"
        except ValueError:
            return f"ERREUR: Strategie invalide '{strategy}'. Valides: exploration, refinement, validation, recovery"


# =============================================================================
# 8.3 LeanSearchPlugin
# =============================================================================

class LeanSearchPlugin:
    """
    Plugin pour la recherche de lemmes dans Mathlib.
    Utilise des patterns connus + verification #check via lean_runner.
    """

    def __init__(self, runner: LeanRunner):
        self._runner = runner
        # Base de lemmes connus (extensible)
        self._known_lemmas = {
            # Arithmetique de base
            "Nat.add_zero": ("n + 0 = n", "Nat"),
            "Nat.zero_add": ("0 + n = n", "Nat"),
            "Nat.add_comm": ("n + m = m + n", "Nat"),
            "Nat.add_assoc": ("(n + m) + k = n + (m + k)", "Nat"),
            "Nat.mul_one": ("n * 1 = n", "Nat"),
            "Nat.one_mul": ("1 * n = n", "Nat"),
            "Nat.mul_comm": ("n * m = m * n", "Nat"),
            "Nat.mul_assoc": ("(n * m) * k = n * (m * k)", "Nat"),
            "Nat.left_distrib": ("n * (m + k) = n * m + n * k", "Nat"),
            "Nat.right_distrib": ("(n + m) * k = n * k + m * k", "Nat"),
            # Logique
            "And.intro": ("a -> b -> a /\\ b", "Logic"),
            "And.left": ("a /\\ b -> a", "Logic"),
            "And.right": ("a /\\ b -> b", "Logic"),
            "Or.inl": ("a -> a \\/ b", "Logic"),
            "Or.inr": ("b -> a \\/ b", "Logic"),
            "Eq.refl": ("a = a", "Logic"),
            "Eq.symm": ("a = b -> b = a", "Logic"),
            "Eq.trans": ("a = b -> b = c -> a = c", "Logic"),
        }

    @kernel_function(
        description="Recherche des lemmes Mathlib pertinents pour un but donne",
        name="search_mathlib_lemmas"
    )
    def search_mathlib_lemmas(self, goal: str, max_results: int = 10) -> str:
        """
        Recherche des lemmes par mots-cles.

        Args:
            goal: Description du but ou mots-cles (ex: "addition commutative")
            max_results: Nombre maximum de resultats

        Returns:
            JSON avec les lemmes trouves
        """
        goal_lower = goal.lower()
        results = []

        # Recherche par mots-cles
        keywords = goal_lower.replace("+", "add").replace("*", "mul").replace("=", "eq").split()

        for name, (statement, namespace) in self._known_lemmas.items():
            score = 0.0
            name_lower = name.lower()

            # Scoring par mots-cles
            for kw in keywords:
                if kw in name_lower:
                    score += 0.3
                if kw in statement.lower():
                    score += 0.2

            # Patterns specifiques
            if "comm" in goal_lower and "comm" in name_lower:
                score += 0.4
            if "assoc" in goal_lower and "assoc" in name_lower:
                score += 0.4
            if "zero" in goal_lower and "zero" in name_lower:
                score += 0.3
            if "distrib" in goal_lower and "distrib" in name_lower:
                score += 0.4

            if score > 0:
                results.append({
                    "name": name,
                    "statement": statement,
                    "namespace": namespace,
                    "relevance": min(score, 1.0)
                })

        # Trier par pertinence
        results.sort(key=lambda x: x["relevance"], reverse=True)
        return json.dumps(results[:max_results], indent=2, ensure_ascii=False)

    @kernel_function(
        description="Verifie qu'un lemme existe et retourne son type via #check",
        name="check_lemma_type"
    )
    def check_lemma_type(self, lemma_name: str) -> str:
        """
        Verifie l'existence d'un lemme via #check.

        Args:
            lemma_name: Nom du lemme (ex: "Nat.add_comm")

        Returns:
            JSON {exists, type, error}
        """
        code = f"#check {lemma_name}"
        result = self._runner.run(code)

        if result.success and not result.errors:
            # Extraire le type de la sortie
            return json.dumps({
                "exists": True,
                "type": result.output.strip(),
                "error": None
            })
        else:
            return json.dumps({
                "exists": False,
                "type": None,
                "error": result.errors or "Lemme non trouve"
            })


# =============================================================================
# 8.4 LeanTacticPlugin


### 8.4-8.5 Plugins de Tactiques et Verification

Les deux plugins restants gerent la **generation de tactiques** et la **verification formelle** :

#### LeanTacticPlugin

- **Responsabilite** : Generer des tactiques Lean adaptees au contexte
- **Methodes exposees** :
  - `generate_tactic()` : Genere une tactique basee sur goal + lemmes + historique
  - `estimate_confidence()` : Estime la probabilite de succes (0.0-1.0)
- **LLM-aware** : Utilise un prompt structure pour le LLM avec exemples de tactiques Lean
- **Strategies** : `exact`, `rw`, `apply`, `simp`, `induction`, `cases`, etc.

#### LeanVerificationPlugin

- **Responsabilite** : Verifier les preuves via compilation Lean
- **Methodes exposees** :
  - `verify_proof()` : Compile le theoreme avec tactiques et retourne succes/echec
  - `parse_lean_errors()` : Parse les messages d'erreur Lean pour feedback agents
- **Detection de completion** : Reconnait "no goals" = preuve complete
- **Gestion d'erreurs** : Extrait type d'erreur (type mismatch, tactic failed, etc.) pour CriticAgent

**Flow typique** :
```
SearchAgent trouve lemmes
   |
   v
TacticAgent genere tactique (via LeanTacticPlugin)
   |
   v
VerifierAgent compile (via LeanVerificationPlugin)
   |
   +-- Success → COMPLETE
   +-- Failure → CriticAgent analyse → retry
```


In [ ]:
# =============================================================================

class LeanTacticPlugin:
    """
    Plugin pour la generation de tactiques.
    Fournit des heuristiques et analyse les echecs.
    """

    def __init__(self):
        # Tactiques par difficulte
        self._tactics = {
            "simple": ["rfl", "trivial", "exact ?_", "assumption"],
            "medium": ["simp", "omega", "decide", "constructor", "intro", "apply"],
            "complex": ["ring", "linarith", "aesop", "induction", "cases", "rcases"]
        }

        # Heuristiques par pattern de but
        self._heuristics = {
            "equality": ["rfl", "exact", "simp", "ring", "omega"],
            "forall": ["intro", "intros", "apply"],
            "exists": ["use", "exists", "exact"],
            "and": ["constructor", "exact And.intro"],
            "or": ["left", "right"],
            "implication": ["intro", "apply", "exact"],
            "nat_arithmetic": ["omega", "simp", "decide"],
            "ring_expression": ["ring", "ring_nf"]
        }

    @kernel_function(
        description="Genere des tactiques appropriees pour un but donne",
        name="generate_tactics"
    )
    def generate_tactics(self, goal: str, context: str = "", difficulty: str = "simple") -> str:
        """
        Genere des tactiques pour le but courant.

        Args:
            goal: Le but Lean a prouver
            context: Contexte additionnel (lemmes disponibles, etc.)
            difficulty: simple, medium, ou complex

        Returns:
            JSON [{tactic, confidence, explanation}]
        """
        suggestions = []
        goal_lower = goal.lower()

        # Detecter le type de but
        detected_patterns = []
        if "=" in goal:
            detected_patterns.append("equality")
        if "forall" in goal_lower or "∀" in goal:
            detected_patterns.append("forall")
        if "exists" in goal_lower or "∃" in goal:
            detected_patterns.append("exists")
        if "/\\" in goal or "∧" in goal or "And" in goal:
            detected_patterns.append("and")
        if "\\/" in goal or "∨" in goal or "Or" in goal:
            detected_patterns.append("or")
        if "->" in goal or "→" in goal:
            detected_patterns.append("implication")
        if any(x in goal_lower for x in ["nat", "n +", "m +", "+ 0", "0 +"]):
            detected_patterns.append("nat_arithmetic")
        if any(x in goal for x in ["*", "+"]) and "=" in goal:
            detected_patterns.append("ring_expression")

        # Collecter les tactiques suggeres
        seen = set()
        for pattern in detected_patterns:
            for tactic in self._heuristics.get(pattern, []):
                if tactic not in seen:
                    seen.add(tactic)
                    confidence = 0.7 if difficulty == "simple" else 0.5
                    suggestions.append({
                        "tactic": tactic,
                        "confidence": confidence,
                        "explanation": f"Pattern detecte: {pattern}"
                    })

        # Ajouter des tactiques de base
        base_tactics = self._tactics.get(difficulty, self._tactics["simple"])
        for tactic in base_tactics[:3]:
            if tactic not in seen:
                suggestions.append({
                    "tactic": tactic,
                    "confidence": 0.3,
                    "explanation": f"Tactique {difficulty} generique"
                })

        # Trier par confiance
        suggestions.sort(key=lambda x: x["confidence"], reverse=True)
        return json.dumps(suggestions[:8], indent=2, ensure_ascii=False)

    @kernel_function(
        description="Analyse un echec de tactique et suggere des alternatives",
        name="analyze_tactic_failure"
    )
    def analyze_tactic_failure(self, failed_tactic: str, error_msg: str) -> str:
        """
        Analyse pourquoi une tactique a echoue.

        Args:
            failed_tactic: La tactique qui a echoue
            error_msg: Message d'erreur Lean

        Returns:
            JSON {diagnosis, alternatives, error_type}
        """
        error_lower = error_msg.lower()
        diagnosis = ""
        alternatives = []
        error_type = "unknown"

        # Classifier l'erreur
        if "unknown identifier" in error_lower or "unknown constant" in error_lower:
            error_type = "unknown_identifier"
            diagnosis = "Lemme ou identifiant non reconnu. Verifier l'import ou le nom."
            alternatives = ["Chercher le bon nom avec #check", "Verifier les imports"]

        elif "type mismatch" in error_lower:
            error_type = "type_mismatch"
            diagnosis = "Les types ne correspondent pas. Verifier les arguments."
            alternatives = ["exact", "apply", "simp"]

        elif "unsolved goals" in error_lower or "goals remain" in error_lower:
            error_type = "unsolved_goals"
            diagnosis = "Des sous-buts restent. La tactique n'a pas complete la preuve."
            alternatives = ["Ajouter d'autres tactiques", "Essayer simp", "Decomposer avec have"]

        elif "tactic failed" in error_lower:
            error_type = "tactic_failed"
            diagnosis = f"La tactique '{failed_tactic}' n'a pas pu s'appliquer."
            # Suggerer des alternatives
            if failed_tactic in ["ring", "linarith"]:
                alternatives = ["omega", "simp", "decide"]
            elif failed_tactic == "simp":
                alternatives = ["simp only", "rfl", "exact"]
            else:
                alternatives = ["simp", "omega", "exact ?_"]

        elif "declaration uses 'sorry'" in error_lower:
            error_type = "sorry"
            diagnosis = "La preuve contient 'sorry' - incomplete."
            alternatives = ["Completer la preuve", "Remplacer sorry par une vraie tactique"]

        else:
            error_type = "other"
            diagnosis = f"Erreur non classifiee: {error_msg[:100]}"
            alternatives = ["Verifier la syntaxe", "Essayer une approche differente"]

        return json.dumps({
            "diagnosis": diagnosis,
            "alternatives": alternatives,
            "error_type": error_type,
            "original_error": error_msg[:200]
        }, indent=2, ensure_ascii=False)


# =============================================================================
# 8.5 LeanVerificationPlugin
# =============================================================================

class LeanVerificationPlugin:
    """
    Plugin pour la verification des preuves avec lean_runner.
    """

    def __init__(self, runner: LeanRunner):
        self._runner = runner

    @kernel_function(
        description="Verifie une preuve complete (theoreme + tactiques)",
        name="verify_proof"
    )
    def verify_proof(self, theorem_statement: str, proof_tactics: str) -> str:
        """
        Verifie un theoreme avec sa preuve.

        Args:
            theorem_statement: L'enonce du theoreme (ex: "theorem add_zero (n : Nat) : n + 0 = n")
            proof_tactics: La preuve (ex: "exact Nat.add_zero n")

        Returns:
            JSON {success, output, errors, exec_time_ms, backend}
        """
        import time

        # Construire le code complet
        if "by" not in proof_tactics and ":=" not in proof_tactics:
            code = f"{theorem_statement} := by {proof_tactics}"
        elif ":=" in proof_tactics:
            code = f"{theorem_statement} {proof_tactics}"
        else:
            code = f"{theorem_statement} := {proof_tactics}"

        start = time.time()
        result = self._runner.run(code)
        exec_time = (time.time() - start) * 1000

        return json.dumps({
            "success": result.success,
            "output": result.output,
            "errors": result.errors,
            "exit_code": result.exit_code,
            "exec_time_ms": round(exec_time, 2),
            "backend": result.backend,
            "code": code
        }, indent=2, ensure_ascii=False)

    @kernel_function(
        description="Verifie une etape de tactique incrementale",
        name="verify_tactic_step"
    )
    def verify_tactic_step(
        self, partial_proof: str, next_tactic: str, theorem_statement: str
    ) -> str:
        """
        Verifie une tactique incrementale.

        Args:
            partial_proof: Les tactiques deja appliquees (separees par ;)
            next_tactic: La prochaine tactique a essayer
            theorem_statement: L'enonce du theoreme

        Returns:
            JSON {tactic_valid, remaining_goals, error, exec_time_ms}
        """
        import time

        # Combiner les tactiques
        if partial_proof:
            all_tactics = f"{partial_proof}; {next_tactic}"
        else:
            all_tactics = next_tactic

        code = f"{theorem_statement} := by {all_tactics}"

        start = time.time()
        result = self._runner.run(code)
        exec_time = (time.time() - start) * 1000

        # Analyser les goals restants
        remaining_goals = None
        if "unsolved goals" in result.errors.lower():
            # Extraire les goals du message d'erreur
            remaining_goals = result.errors

        return json.dumps({
            "tactic_valid": result.success or "unsolved goals" not in result.errors.lower(),
            "remaining_goals": remaining_goals,
            "error": result.errors if not result.success else None,
            "exec_time_ms": round(exec_time, 2),
            "applied_tactics": all_tactics
        }, indent=2, ensure_ascii=False)


# =============================================================================
# Test des Plugins
# =============================================================================

print("\n=== Test des Plugins ===")

# Creer l'etat et le runner
test_state = ProofState(theorem_statement="theorem test_add (n : Nat) : n + 0 = n")
runner = LeanRunner(backend="subprocess", timeout=30)

# Instancier les plugins
state_plugin = ProofStateManagerPlugin(test_state)
search_plugin = LeanSearchPlugin(runner)
tactic_plugin = LeanTacticPlugin()
verif_plugin = LeanVerificationPlugin(runner)

# Test 1: Recherche de lemmes
print("\n1. Recherche de lemmes pour 'addition zero':")
lemmas = search_plugin.search_mathlib_lemmas("addition zero", max_results=3)
print(lemmas)

# Test 2: Generation de tactiques
print("\n2. Tactiques pour 'n + 0 = n':")
tactics = tactic_plugin.generate_tactics("n + 0 = n", difficulty="simple")
print(tactics)

# Test 3: Verification avec lean_runner
print("\n3. Verification d'une preuve:")
result = verif_plugin.verify_proof("theorem test_rfl : 2 + 2 = 4", "rfl")
print(result)

# Test 4: Plugin StateManager
print("\n4. Ajout via StateManagerPlugin:")
print(state_plugin.add_discovered_lemma("Nat.add_zero", "n + 0 = n", "Nat", 0.9))
print(state_plugin.get_proof_state(summarize=True))


### 8.6 Definition des 5 Agents Specialises

Le systeme multi-agents comprend 5 roles distincts:

| Agent | Role | Plugins | Delegation |
|-------|------|---------|------------|
| **SearchAgent** | Recherche lemmes Mathlib | LeanSearch, StateManager | TacticAgent si lemmes trouves |
| **TacticAgent** | Generation tactiques | LeanTactic, StateManager | VerifierAgent pour validation |
| **VerifierAgent** | Verification Lean | LeanVerification, StateManager | CriticAgent si echec |
| **CriticAgent** | Analyse echecs | LeanTactic, StateManager | Redirection selon erreur |
| **CoordinatorAgent** | Supervision globale | StateManager | Gestion des blocages |

**Pattern cle**: Chaque agent designe explicitement le suivant via `designate_next_agent()`.

## 🤖 Création des Agents Semantic Kernel

### Anatomie d'un agent

Chaque agent a :

1. **Un nom** : "SearchAgent", "TacticAgent", etc.
2. **Des instructions** : Prompt système qui définit son rôle
3. **Des plugins** : Fonctions qu'il peut appeler (via StatePlugin)
4. **Un modèle LLM** : GPT-5.2, Claude, etc.

### Exemple : SearchAgent

```python
search_agent = kernel.add_agent(
    name="SearchAgent",
    instructions="""Tu es un expert en recherche de lemmes Mathlib.
    Ton rôle : Trouver les lemmes pertinents pour le but actuel.
    Délègue à TacticAgent une fois les lemmes trouvés.""",
    plugins=[state_plugin]
)
```

### Instructions : Le "métier" de l'agent

Les instructions définissent :

- **Responsabilité** : "Recherche de lemmes" vs "Génération de tactiques"
- **Critères de succès** : "Trouver au moins 2 lemmes pertinents"
- **Délégation** : "Quand déléguer à un autre agent ?"

**Principe clé** : Instructions précises → Comportement prévisible

### Pattern : Stratégies basées sur l'état

Au lieu de coder en dur "SearchAgent → TacticAgent", on utilise :

```python
def select_next_agent(state: ProofState) -> str:
    if state.phase == ProofPhase.SEARCH:
        return "SearchAgent"
    elif state.phase == ProofPhase.TACTIC_GEN:
        return "TacticAgent"
    # ...
```

**Avantage** : Orchestration dynamique basée sur l'état réel de la preuve.

In [ ]:
# =============================================================================
# Section 8.6 - Definition des 5 Agents Specialises avec Semantic Kernel
# =============================================================================
# Utilise ChatCompletionAgent de Semantic Kernel avec FunctionChoiceBehavior.Auto()

import os
import asyncio
from typing import Dict, Any, Optional

# --- Instructions des Agents ---

SEARCH_AGENT_INSTRUCTIONS = """
Tu es l'agent de RECHERCHE de lemmes pour le theorem proving en Lean 4.

TON ROLE UNIQUE:
- Chercher des lemmes Mathlib pertinents pour le theoreme courant
- Identifier les lemmes qui peuvent aider a la preuve
- Enregistrer les lemmes trouves dans l'etat partage

WORKFLOW:
1. Lis l'etat avec get_proof_state() pour comprendre le theoreme
2. Utilise search_mathlib_lemmas() avec des mots-cles pertinents
3. Verifie les lemmes prometteurs avec check_lemma_type()
4. Enregistre les lemmes utiles avec add_discovered_lemma()
5. Delegue a TacticAgent quand tu as trouve des lemmes

IMPORTANT:
- Cherche des lemmes LIES au but (egalites, arithmetique, logique)
- Delegation: Apres avoir trouve au moins 2-3 lemmes, delegue a TacticAgent
- Si aucun lemme pertinent, delegue quand meme a TacticAgent
"""

TACTIC_AGENT_INSTRUCTIONS = """
Tu es l'agent de GENERATION DE TACTIQUES pour le theorem proving en Lean 4.

TON ROLE UNIQUE:
- Generer des sequences de tactiques Lean pour prouver le but
- Utiliser les lemmes trouves par SearchAgent
- Proposer des tactiques avec niveau de confiance

WORKFLOW:
1. Lis l'etat avec get_proof_state() pour voir le theoreme et les lemmes
2. Utilise generate_tactics() pour obtenir des suggestions
3. Enregistre ta meilleure tentative avec log_tactic_attempt()
4. Delegue a VerifierAgent pour verification

STRATEGIES DE TACTIQUES (par difficulte):
- SIMPLE: rfl, trivial, exact, assumption
- MEDIUM: simp, omega, constructor, intro, apply
- COMPLEX: ring, linarith, induction, cases

IMPORTANT:
- Commence par les tactiques simples (rfl, exact)
- Utilise les lemmes trouves par SearchAgent (exact Nat.add_zero n)
- Delegation: TOUJOURS deleguer a VerifierAgent apres avoir propose
"""

VERIFIER_AGENT_INSTRUCTIONS = """
Tu es l'agent de VERIFICATION pour le theorem proving en Lean 4.

TON ROLE UNIQUE:
- Verifier les tactiques proposees avec le compilateur Lean
- Enregistrer les resultats de verification
- Determiner si la preuve est complete ou s'il faut continuer

WORKFLOW:
1. Lis l'etat avec get_proof_state() pour voir la derniere tactique
2. Utilise verify_proof() pour tester la preuve
3. Enregistre le resultat avec add_verification_result()
4. Si succes: set_proof_complete() et termine
5. Si echec: delegue a CriticAgent pour analyse

IMPORTANT:
- Teste TOUJOURS la derniere tactique proposee
- Si la preuve compile sans erreur, utilise set_proof_complete()
- Si echec, enregistre l'erreur et delegue a CriticAgent
"""

CRITIC_AGENT_INSTRUCTIONS = """
Tu es l'agent CRITIQUE pour le theorem proving en Lean 4.

TON ROLE UNIQUE:
- Analyser les echecs de verification
- Diagnostiquer les erreurs Lean
- Orienter vers la bonne strategie de correction

WORKFLOW:
1. Lis l'etat avec get_proof_state() pour voir les echecs recents
2. Utilise analyze_tactic_failure() pour comprendre l'erreur
3. Decide quelle direction prendre:
   - "unknown identifier" -> delegue a SearchAgent
   - "type mismatch" ou "tactic failed" -> delegue a TacticAgent
   - Echecs repetes (>3) -> delegue a CoordinatorAgent

IMPORTANT:
- Analyse les 3 derniers echecs pour detecter des patterns
- Si >3 echecs similaires, delegue a CoordinatorAgent
"""

COORDINATOR_AGENT_INSTRUCTIONS = """
Tu es l'agent COORDINATEUR (superviseur) pour le theorem proving en Lean 4.

TON ROLE UNIQUE:
- Superviser l'ensemble de la session de preuve
- Debloquer les situations cycliques
- Ajuster la strategie globale

QUAND TU INTERVIENS:
- Appele par CriticAgent apres echecs repetes
- Appele si max_iterations approche
- Appele pour decisions strategiques majeures

IMPORTANT:
- Tu es le dernier recours, prends des decisions audacieuses
- Si >40 iterations, suggere de simplifier le theoreme
"""

# =============================================================================
# Detection de Semantic Kernel
# =============================================================================

SK_AVAILABLE = False
try:
    from semantic_kernel import Kernel
    from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
    from semantic_kernel.agents.strategies import (
        KernelFunctionSelectionStrategy,
        KernelFunctionTerminationStrategy,
    )
    from semantic_kernel.agents.strategies.selection.sequential_selection_strategy import SequentialSelectionStrategy
    from semantic_kernel.agents.strategies.termination.default_termination_strategy import DefaultTerminationStrategy
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
    from semantic_kernel.connectors.ai import FunctionChoiceBehavior
    from semantic_kernel.functions import KernelFunctionFromPrompt, KernelArguments
    from semantic_kernel.contents import ChatHistoryTruncationReducer
    from semantic_kernel.agents.strategies.selection.selection_strategy import SelectionStrategy
    from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
    from semantic_kernel.contents.chat_message_content import ChatMessageContent
    from pydantic import PrivateAttr
    SK_AVAILABLE = True
    print("Semantic Kernel disponible - utilisation de ChatCompletionAgent")
except ImportError as e:
    print(f"Semantic Kernel non disponible: {e}")
    print("Installation: pip install semantic-kernel")

# =============================================================================
# Mode Simulation (fallback si SK non disponible)
# =============================================================================

class SimpleAgent:
    """
    Agent simplifie pour simulation ou fallback.
    Utilise directement l'API OpenAI si SK n'est pas disponible.
    """

    def __init__(
        self,
        name: str,
        instructions: str,
        plugins: Dict[str, Any],
        use_simulation: bool = True
    ):
        self.name = name
        self.instructions = instructions
        self.plugins = plugins
        self.use_simulation = use_simulation
        self._openai_client = None

        # Initialiser le client OpenAI si mode reel
        if not use_simulation:
            try:
                from openai import OpenAI
                api_key = os.getenv("OPENAI_API_KEY")
                if api_key and len(api_key) > 10 and not api_key.startswith("sk-..."):
                    self._openai_client = OpenAI(api_key=api_key)
            except ImportError:
                pass

    def _build_openai_tools(self) -> list:
        """Construit les outils au format OpenAI function calling."""
        import inspect
        tools = []
        for plugin_name, plugin in self.plugins.items():
            for attr_name in dir(plugin):
                attr = getattr(plugin, attr_name)
                if not callable(attr):
                    continue
                # Supporter les deux décorateurs
                is_sk_func = hasattr(attr, '_sk_function') or hasattr(attr, '__kernel_function__')
                if not is_sk_func:
                    continue

                sig = inspect.signature(attr)
                properties = {}
                required = []
                for param_name, param in sig.parameters.items():
                    if param_name == 'self':
                        continue
                    param_type = "string"
                    if param.annotation != inspect.Parameter.empty:
                        if param.annotation == bool:
                            param_type = "boolean"
                        elif param.annotation in (int, float):
                            param_type = "number"
                    properties[param_name] = {
                        "type": param_type,
                        "description": f"Parameter {param_name}"
                    }
                    if param.default == inspect.Parameter.empty:
                        required.append(param_name)

                # Obtenir nom et description
                if hasattr(attr, '__kernel_function_name__'):
                    func_name = attr.__kernel_function_name__
                    func_desc = getattr(attr, "__kernel_function_description__", "")
                elif hasattr(attr, '_sk_name'):
                    func_name = attr._sk_name
                    func_desc = getattr(attr, "_sk_description", "")
                else:
                    func_name = attr_name
                    func_desc = ""

                tools.append({
                    "type": "function",
                    "function": {
                        "name": f"{plugin_name}__{func_name}",
                        "description": func_desc,
                        "parameters": {
                            "type": "object",
                            "properties": properties,
                            "required": required
                        }
                    }
                })
        return tools

    def _execute_tool_call(self, tool_name: str, arguments: dict) -> str:
        """Execute un appel de fonction sur un plugin."""
        parts = tool_name.split("__", 1)
        if len(parts) != 2:
            return f"Erreur: format invalide: {tool_name}"

        plugin_name, func_name = parts
        plugin = self.plugins.get(plugin_name)
        if not plugin:
            return f"Erreur: plugin {plugin_name} non trouve"

        for attr_name in dir(plugin):
            attr = getattr(plugin, attr_name)
            if not callable(attr):
                continue
            is_sk = hasattr(attr, '_sk_function') or hasattr(attr, '__kernel_function__')
            if not is_sk:
                continue

            if hasattr(attr, '__kernel_function_name__'):
                name = attr.__kernel_function_name__
            elif hasattr(attr, '_sk_name'):
                name = attr._sk_name
            else:
                name = attr_name

            if name == func_name:
                try:
                    result = attr(**arguments)
                    return str(result)
                except Exception as e:
                    return f"Erreur {func_name}: {e}"

        return f"Erreur: {func_name} non trouve dans {plugin_name}"

    def invoke(self, message: str, state: ProofState) -> str:
        """Execute l'agent sur un message."""
        state.increment_iteration()

        if self.use_simulation or not self._openai_client:
            return self._simulate_response(message, state)
        else:
            return self._call_llm(message, state)

    def _simulate_response(self, message: str, state: ProofState) -> str:
        """Simulation de l'agent (sans appels LLM)."""
        if self.name == "SearchAgent":
            search = self.plugins.get("search")
            state_mgr = self.plugins.get("state")
            if search and state_mgr:
                lemmas_json = search.search_mathlib_lemmas(state.theorem_goal or "addition", max_results=3)
                lemmas = json.loads(lemmas_json)
                for lemma in lemmas[:2]:
                    state_mgr.add_discovered_lemma(lemma["name"], lemma["statement"], lemma["namespace"], lemma["relevance"])
                state_mgr.designate_next_agent("TacticAgent")
                return f"[SearchAgent] Trouves {len(lemmas[:2])} lemmes. Delegation a TacticAgent."

        elif self.name == "TacticAgent":
            tactic = self.plugins.get("tactic")
            state_mgr = self.plugins.get("state")
            if tactic and state_mgr:
                tactics_json = tactic.generate_tactics(state.theorem_goal or "n + 0 = n", difficulty="simple")
                tactics = json.loads(tactics_json)
                if tactics:
                    best = tactics[0]
                    state_mgr.log_tactic_attempt(best["tactic"], state.theorem_goal or "", best["confidence"], best["explanation"])
                state_mgr.designate_next_agent("VerifierAgent")
                return f"[TacticAgent] Propose: {tactics[0]['tactic'] if tactics else 'rfl'}. Delegation a VerifierAgent."

        elif self.name == "VerifierAgent":
            verif = self.plugins.get("verification")
            state_mgr = self.plugins.get("state")
            if verif and state_mgr and state.tactics_history:
                last_tactic = state.tactics_history[-1]
                result_json = verif.verify_proof(state.theorem_statement, last_tactic.tactic)
                result = json.loads(result_json)
                state_mgr.add_verification_result(
                    last_tactic.id, result["success"], result["output"], result["errors"],
                    "", result["exec_time_ms"]
                )
                if result["success"]:
                    state_mgr.set_proof_complete(last_tactic.tactic)
                    return f"[VerifierAgent] SUCCES! Preuve: {last_tactic.tactic}"
                else:
                    state_mgr.designate_next_agent("CriticAgent")
                    return f"[VerifierAgent] Echec: {result['errors'][:100]}. Delegation a CriticAgent."

        elif self.name == "CriticAgent":
            tactic = self.plugins.get("tactic")
            state_mgr = self.plugins.get("state")
            if tactic and state_mgr:
                failures = state.get_recent_failures(3)
                if failures:
                    _, last_verif = failures[0]
                    analysis_json = tactic.analyze_tactic_failure("unknown", last_verif.errors)
                    analysis = json.loads(analysis_json)
                    if analysis["error_type"] == "unknown_identifier":
                        state_mgr.designate_next_agent("SearchAgent")
                        return f"[CriticAgent] Identifiant inconnu. Retour a SearchAgent."
                    elif len(failures) > 3:
                        state_mgr.designate_next_agent("CoordinatorAgent")
                        return f"[CriticAgent] Trop d'echecs. Appel CoordinatorAgent."
                    else:
                        state_mgr.designate_next_agent("TacticAgent")
                        return f"[CriticAgent] Essayer d'autres tactiques. -> TacticAgent."
                state_mgr.designate_next_agent("TacticAgent")
                return "[CriticAgent] Pas d'echecs recents. -> TacticAgent."

        elif self.name == "CoordinatorAgent":
            state_mgr = self.plugins.get("state")
            if state_mgr:
                if state.iteration_count > 30:
                    state_mgr.set_proof_strategy("recovery")
                    return "[CoordinatorAgent] Mode recovery active."
                else:
                    state_mgr.set_proof_strategy("refinement")
                    state_mgr.designate_next_agent("TacticAgent")
                    return "[CoordinatorAgent] Strategie refinement. -> TacticAgent."

        return f"[{self.name}] Action simulee."

    def _call_llm(self, message: str, state: ProofState) -> str:
        """Appelle le LLM OpenAI avec function calling."""
        state_summary = json.dumps(state.get_state_snapshot(summarize=True), indent=2)
        tools = self._build_openai_tools()

        nl = chr(10)
        user_content = f"ETAT ACTUEL:{nl}{state_summary}{nl}{nl}TACHE:{nl}{message}"
        messages = [
            {"role": "system", "content": self.instructions},
            {"role": "user", "content": user_content}
        ]

        max_tool_calls = 10
        tool_results = []

        for iteration in range(max_tool_calls):
            try:
                model = os.getenv("OPENAI_CHAT_MODEL_ID", "gpt-5.2")
                use_mct = any(model.startswith(p) for p in ('gpt-4.5', 'gpt-5', 'o1', 'o3'))
                token_param = {"max_completion_tokens": 1000} if use_mct else {"max_tokens": 1000}

                response = self._openai_client.chat.completions.create(
                    model=model,
                    messages=messages,
                    tools=tools if tools else None,
                    tool_choice="auto" if tools else None,
                    temperature=0.3,
                    **token_param
                )

                assistant_message = response.choices[0].message

                if assistant_message.tool_calls:
                    messages.append(assistant_message.model_dump())

                    for tool_call in assistant_message.tool_calls:
                        func_name = tool_call.function.name
                        try:
                            arguments = json.loads(tool_call.function.arguments)
                        except json.JSONDecodeError:
                            arguments = {}

                        result = self._execute_tool_call(func_name, arguments)
                        tool_results.append(func_name.split("__")[-1])

                        messages.append({
                            "role": "tool",
                            "tool_call_id": tool_call.id,
                            "content": result
                        })
                else:
                    final_response = assistant_message.content or "(pas de reponse)"
                    if tool_results:
                        actions = ", ".join(tool_results[:5])
                        final_response = f"Actions: {actions}{nl}{final_response}"
                    return f"[{self.name}] {final_response}"

            except Exception as e:
                return f"[{self.name}] Erreur LLM: {e}"

        actions = ", ".join(tool_results[:5])
        return f"[{self.name}] Max tool calls. Actions: {actions}"


### Patterns de Delegation Multi-Agents

Les instructions ci-dessus definissent les **regles de delegation** entre agents :

| Agent | Role | Delegue vers |
|-------|------|-------------|
| **SearchAgent** | Recherche lemmes Mathlib | TacticAgent (si lemmes trouves) |
| **TacticAgent** | Genere tactiques Lean | VerifierAgent (toujours) |
| **VerifierAgent** | Verifie preuve formelle | CriticAgent (si echec) / COMPLETE (si succes) |
| **CriticAgent** | Analyse erreurs | SearchAgent (retry) / CoordinatorAgent (si bloque) |
| **CoordinatorAgent** | Re-orchestre strategie | SearchAgent (nouvelle strategie) |

**Flow nominal** (preuve simple) :
```
SearchAgent → TacticAgent → VerifierAgent → COMPLETE
```

**Flow avec echec** (preuve complexe) :
```
SearchAgent → TacticAgent → VerifierAgent (FAIL)
   ↓
CriticAgent analyse erreur
   ↓
   +-- Erreur simple → SearchAgent (retry avec nouvelles contraintes)
   +-- Erreur complexe → CoordinatorAgent (changement strategie)
```

**Note critique** : Les demos actuelles (DEMO_1-3) sont trop triviales et ne declenchent JAMAIS CriticAgent ni CoordinatorAgent. DEMO_4 (list_length_append) devrait necessiter induction et potentiellement trigger ces agents.


### Quand CriticAgent et CoordinatorAgent Interviennent

#### CriticAgent : Analyse d'Echecs de Tactiques

**Declenche par VerifierAgent quand** :
- `verify_proof()` retourne `success=False`
- Erreur Lean detectee : type mismatch, tactic failed, unknown identifier
- Preuve incomplete apres application de tactique

**Responsabilites** :
1. Parser l'erreur Lean (extraire type, message, contexte)
2. Identifier la cause (lemme incorrect, tactique inadequate, goal mal compris)
3. Proposer correction :
   - Erreur simple (lemme manquant) → Delegue SearchAgent avec contraintes
   - Erreur complexe (strategie incorrecte) → Delegue CoordinatorAgent

**Exemple d'intervention** :
```
[Tour 5] VerifierAgent: FAIL - "type mismatch, expected Nat but got Bool"
[Tour 6] CriticAgent: "TacticAgent a applique 'exact lemma_bool' mais goal attend Nat.
                       SearchAgent doit chercher lemmes avec type Nat -> Nat."
[Tour 7] SearchAgent: Recherche lemmes type-aware...
```

**Pourquoi absent des demos actuelles** :
- DEMO_1-3 : Lemmes Mathlib correspondent exactement au goal
- Pas de type mismatch, pas de tactic failure
- VerifierAgent retourne success au premier essai

#### CoordinatorAgent : Re-Orchestration Strategique

**Declenche par CriticAgent quand** :
- Echecs multiples consecutifs (3+ iterations sans progres)
- Strategie actuelle bloquee (EXPLORATION → REFINEMENT → toujours FAIL)
- Pattern d'erreur complexe (induction necessaire mais pas tentee)

**Responsabilites** :
1. Analyser historique complet (ProofState.snapshots)
2. Identifier pattern d'echec (loop, strategie inadequate)
3. Changer strategie globale :
   - EXPLORATION → VALIDATION (essayer preuves directes)
   - REFINEMENT → RECOVERY (backtrack + nouvelle approche)
4. Reset partiel de ProofState (clear failed tactics, keep lemmas)

**Exemple d'intervention** :
```
[Tour 8] CriticAgent: "Echec 3x consecutif avec meme lemme. Strategie bloquee."
[Tour 9] CoordinatorAgent: "Detection pattern: goal necessite induction mais pas tentee.
                            Changement strategie: EXPLORATION → RECOVERY.
                            Ajout contrainte: TacticAgent DOIT considerer 'induction'."
[Tour 10] SearchAgent: Recherche lemmes inductifs...
```

**Pourquoi absent des demos actuelles** :
- DEMO_1-3 : Pas d'echecs, donc CriticAgent jamais declenche
- DEMO_4 (list_length_append) : **DEVRAIT** declencher si :
  - Lemme direct `List.length_append` pas trouve
  - TacticAgent essaie `rw` ou `simp` sans induction → echec
  - CriticAgent detecte besoin d'induction
  - CoordinatorAgent change strategie vers RECOVERY

#### Activation des Agents Critiques

| Scenario | SearchAgent | TacticAgent | VerifierAgent | CriticAgent | CoordinatorAgent |
|----------|-------------|-------------|---------------|-------------|------------------|
| **Preuve triviale** (rfl) | ✗ | ✓ | ✓ | ✗ | ✗ |
| **Lemme direct trouve** (exact) | ✓ | ✓ | ✓ | ✗ | ✗ |
| **Lemme incorrect** (type mismatch) | ✓ | ✓ | ✓ | ✓ | ✗ |
| **Tactique echoue 1x** (retry) | ✓ | ✓ | ✓ | ✓ | ✗ |
| **Tactique echoue 3x** (bloque) | ✓ | ✓ | ✓ | ✓ | ✓ |
| **Induction necessaire** | ✓ | ✓ | ✓ | ✓ | ✓ |

**Conclusion** : Pour tester CriticAgent et CoordinatorAgent, nous devons utiliser des theoremes ou :
1. Mathlib n'a PAS de lemme direct exact match
2. Preuve necessite composition de tactiques (rw + simp + induction)
3. Premiere tentative echoue et necessite correction

**DEMO_4 (list_length_append) est concu pour ca** - mais seulement si on desactive l'acces au lemme `List.length_append` de Mathlib.


In [ ]:


# =============================================================================
# Factory pour creer les agents (SK ou fallback)
# =============================================================================

def create_agents(
    plugins: Dict[str, Any],
    state: ProofState,
    use_sk: bool = True,
    use_simulation: bool = False
) -> Dict[str, Any]:
    """
    Cree les 5 agents specialises.

    Args:
        plugins: Dictionnaire des plugins SK
        state: Etat partage de la preuve
        use_sk: Utiliser Semantic Kernel si disponible
        use_simulation: Mode simulation (sans appels LLM)

    Returns:
        Dictionnaire {nom_agent: agent}
    """
    if use_sk and SK_AVAILABLE and not use_simulation:
        return _create_sk_agents(plugins, state)
    else:
        return _create_simple_agents(plugins, use_simulation)


def _create_simple_agents(plugins: Dict[str, Any], use_simulation: bool) -> Dict[str, Any]:
    """Cree les agents en mode fallback/simulation."""
    return {
        "SearchAgent": SimpleAgent("SearchAgent", SEARCH_AGENT_INSTRUCTIONS, plugins, use_simulation),
        "TacticAgent": SimpleAgent("TacticAgent", TACTIC_AGENT_INSTRUCTIONS, plugins, use_simulation),
        "VerifierAgent": SimpleAgent("VerifierAgent", VERIFIER_AGENT_INSTRUCTIONS, plugins, use_simulation),
        "CriticAgent": SimpleAgent("CriticAgent", CRITIC_AGENT_INSTRUCTIONS, plugins, use_simulation),
        "CoordinatorAgent": SimpleAgent("CoordinatorAgent", COORDINATOR_AGENT_INSTRUCTIONS, plugins, use_simulation),
    }


def _create_sk_agents(plugins: Dict[str, Any], state: ProofState) -> Dict[str, Any]:
    """
    Cree les agents avec Semantic Kernel ChatCompletionAgent.

    Utilise:
    - OpenAIChatCompletion pour le service LLM
    - FunctionChoiceBehavior.Auto() pour le function calling automatique
    - Les plugins existants sont passes aux agents
    """
    # Creer le kernel et le service
    kernel = Kernel()
    api_key = os.getenv("OPENAI_API_KEY")
    model = os.getenv("OPENAI_CHAT_MODEL_ID", "gpt-5.2")

    service = OpenAIChatCompletion(
        service_id="openai",
        ai_model_id=model,
        api_key=api_key
    )
    kernel.add_service(service)

    # Ajouter les plugins au kernel
    for plugin_name, plugin in plugins.items():
        kernel.add_plugin(plugin, plugin_name=plugin_name)

    # Configuration pour auto function calling
    settings = kernel.get_prompt_execution_settings_from_service_id(service_id="openai")
    settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

    # Creer les agents
    agents = {}
    agent_configs = [
        ("SearchAgent", SEARCH_AGENT_INSTRUCTIONS),
        ("TacticAgent", TACTIC_AGENT_INSTRUCTIONS),
        ("VerifierAgent", VERIFIER_AGENT_INSTRUCTIONS),
        ("CriticAgent", CRITIC_AGENT_INSTRUCTIONS),
        ("CoordinatorAgent", COORDINATOR_AGENT_INSTRUCTIONS),
    ]

    for name, instructions in agent_configs:
        agents[name] = ChatCompletionAgent(
            kernel=kernel,
            name=name,
            instructions=instructions,
            arguments=KernelArguments(settings=settings)
        )

    print(f"Crees {len(agents)} agents SK avec modele {model}")
    return agents


# =============================================================================
# Test des Agents
# =============================================================================

print("\n=== Test des Agents ===")

# Creer l'environnement
test_state = ProofState(
    theorem_statement="theorem add_zero (n : Nat) : n + 0 = n",
    current_goal="n + 0 = n"
)
runner = LeanRunner(backend="subprocess", timeout=30)

# Creer les plugins
plugins = {
    "state": ProofStateManagerPlugin(test_state),
    "search": LeanSearchPlugin(runner),
    "tactic": LeanTacticPlugin(),
    "verification": LeanVerificationPlugin(runner)
}

# Mode de fonctionnement
USE_SK = SK_AVAILABLE and os.getenv("OPENAI_API_KEY")
USE_SIMULATION = not USE_SK  # Simulation si SK non disponible ou pas de cle API

print(f"Mode: {'Semantic Kernel' if USE_SK else 'Simulation'}")

# Creer les agents
agents = create_agents(plugins, test_state, use_sk=USE_SK, use_simulation=USE_SIMULATION)

# Test rapide en mode simulation
if USE_SIMULATION:
    print("\nTest SearchAgent (simulation):")
    response = agents["SearchAgent"].invoke("Trouve des lemmes pour n + 0 = n", test_state)
    print(response)
    print(f"Etat apres SearchAgent:\n{test_state}")


### Vue d'Ensemble des 5 Agents Specialises

La fonction `create_agents()` instancie les 5 agents avec :
- **Instructions** : Prompts systemiques definissant role et regles de delegation
- **Plugins** : Fonctions exposees (search, tactic generation, verification, etc.)
- **Modele LLM** : gpt-5.2 (ou simulation si mode LLM desactive)

#### Signatures des agents

```python
SearchAgent(
    plugins=[lean_search_plugin, state_plugin],
    instructions="Trouve lemmes Mathlib pertinents..."
)

TacticAgent(
    plugins=[tactic_plugin, state_plugin],
    instructions="Genere tactiques Lean avec confiance..."
)

VerifierAgent(
    plugins=[verification_plugin, state_plugin],
    instructions="Compile et verifie preuves formelles..."
)

CriticAgent(
    plugins=[state_plugin],
    instructions="Analyse echecs et propose corrections..."
)

CoordinatorAgent(
    plugins=[state_plugin],
    instructions="Re-orchestre strategie globale..."
)
```

**Pattern cle** : Chaque agent n'a acces qu'aux plugins dont il a besoin (principe de moindre privilege). Le `state_plugin` est partage par tous pour consulter/modifier ProofState.


### 8.7 Strategies d'Orchestration

L'orchestration determine comment les agents sont selectionnes et quand la conversation se termine.

**DelegatingSelectionStrategy** (Pattern recommande):
- Chaque agent designe explicitement le suivant via `designate_next_agent()`
- Si aucune designation, utilise un agent par defaut (CoordinatorAgent)

**ProofCompleteTermination**:
- Termine si `proof_complete == True`
- Termine si `iteration_count >= max_iterations`

### 8.8 Demonstration Complete

Cette demonstration montre le workflow multi-agents complet:
1. **CoordinatorAgent** initialise la session
2. **SearchAgent** recherche les lemmes pertinents
3. **TacticAgent** propose des tactiques
4. **VerifierAgent** verifie avec Lean
5. **CriticAgent** intervient en cas d'echec

## 🎭 Orchestration Multi-Agents

### Le problème de l'orchestration

Avec 5 agents, qui parle quand ? Deux approches :

1. **Statique** : SearchAgent → TacticAgent → VerifierAgent (toujours)
   - Simple mais rigide
   - Pas de backtracking

2. **Dynamique** : Décisions basées sur l'état de la preuve
   - Flexible mais complexe
   - Permet le backtracking et la récupération d'erreur

**Nous utilisons l'approche dynamique.**

### Stratégies d'orchestration

#### ProofSelectionStrategy

Décide **quel agent agit** à chaque tour :

```python
class ProofSelectionStrategy:
    def select_next_agent(self, state: ProofState, agents: List[str]) -> str:
        if state.phase == ProofPhase.INIT:
            return "CoordinatorAgent"
        elif state.phase == ProofPhase.SEARCH:
            return "SearchAgent"
        # ...
```

#### ProofTerminationStrategy

Décide **quand arrêter** la session :

```python
class ProofTerminationStrategy:
    def should_terminate(self, state: ProofState, iteration: int) -> Tuple[bool, str]:
        if state.phase == ProofPhase.COMPLETE:
            return (True, "Preuve complète!")
        if iteration >= max_iterations:
            return (True, "Timeout atteint")
        # ...
```

### Boucle principale

```python
while not should_terminate:
    # 1. Sélectionner agent
    agent_name = selection_strategy.select_next_agent(state, agents)

    # 2. Exécuter agent (appelle le LLM)
    response = agent.chat(f"Phase: {state.phase}, Goal: {state.current_goal}")

    # 3. L'agent appelle des plugins (modifie l'état)
    # Exemple: log_tactic_attempt("rw [Nat.add_zero]")

    # 4. Mettre à jour phase selon résultat
    update_phase(state)

    # 5. Vérifier condition de terminaison
    should_terminate, reason = termination_strategy.should_terminate(state, iteration)
```

### Snapshots : Observer l'orchestration

À chaque tour, on sauvegarde :

```json
{
  "iteration": 5,
  "agent": "TacticAgent",
  "phase_before": "SEARCH",
  "phase_after": "TACTIC_GEN",
  "action": "Generated tactic: rw [Nat.add_zero]",
  "state_snapshot": {...}
}
```

**Utilité** : Voir exactement quelle décision chaque agent a prise.

In [107]:
# =============================================================================
# Section 8.7 - Strategies d'Orchestration (Pattern Argument_Analysis)
# =============================================================================
# Strategies personnalisees basees sur l'etat partage (pas sur l'historique)

# Fix for Jupyter event loop
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    pass

import logging
from typing import Dict, Any, List, Optional

# =============================================================================
# ProofSelectionStrategy - Selection basee sur l'etat partage
# =============================================================================

class ProofSelectionStrategy(SelectionStrategy):
    """
    Strategie de selection d'agent basee sur l'etat partage ProofState.
    Lit state.consume_next_agent_designation() au lieu de parser l'historique.
    Pattern inspire de Argument_Analysis/DelegatingSelectionStrategy.
    """

    _agents_map: Dict[str, Any] = PrivateAttr()
    _state: 'ProofState' = PrivateAttr()
    _default_agent_name: str = PrivateAttr()
    _logger: logging.Logger = PrivateAttr()

    def __init__(self, agents: List[Any], state: 'ProofState',
                 default_agent_name: str = "SearchAgent"):
        super().__init__()
        if not hasattr(state, 'consume_next_agent_designation'):
            raise TypeError("state doit avoir consume_next_agent_designation()")
        self._agents_map = {agent.name: agent for agent in agents}
        self._state = state
        self._default_agent_name = default_agent_name
        self._logger = logging.getLogger("ProofSelectionStrategy")

    async def next(self, agents: List[Any], history: List[ChatMessageContent]) -> Any:
        """Selectionne le prochain agent base sur l'etat partage."""
        # 1. Verifier designation explicite dans l'etat
        try:
            designated = self._state.consume_next_agent_designation()
            if designated:
                self._logger.info(f"Designation explicite: {designated}")
                if designated in self._agents_map:
                    return self._agents_map[designated]
                self._logger.warning(f"Agent '{designated}' non trouve!")
        except Exception as e:
            self._logger.error(f"Erreur lecture designation: {e}")

        # 2. Logique basee sur la phase
        try:
            phase = self._state.phase.value
            if phase == "search":
                return self._agents_map.get("TacticAgent", self._agents_map.get(self._default_agent_name))
            elif phase == "generate":
                return self._agents_map.get("VerifierAgent", self._agents_map.get(self._default_agent_name))
            elif phase == "verify":
                if self._state.last_error:
                    return self._agents_map.get("CriticAgent", self._agents_map.get(self._default_agent_name))
                return self._agents_map.get("CoordinatorAgent", self._agents_map.get(self._default_agent_name))
            elif phase == "analyze":
                return self._agents_map.get("SearchAgent", self._agents_map.get(self._default_agent_name))
        except Exception as e:
            self._logger.error(f"Erreur logique phase: {e}")

        return self._agents_map.get(self._default_agent_name)

    async def reset(self) -> None:
        try:
            self._state.consume_next_agent_designation()
        except:
            pass


# =============================================================================
# ProofTerminationStrategy - Terminaison basee sur l'etat partage
# =============================================================================

class ProofTerminationStrategy(TerminationStrategy):
    """
    Strategie de terminaison basee sur l'etat partage ProofState.
    Verifie state.proof_complete au lieu d'analyser les messages.
    """

    _state: 'ProofState' = PrivateAttr()
    _max_iterations: int = PrivateAttr()
    _logger: logging.Logger = PrivateAttr()

    def __init__(self, state: 'ProofState', max_iterations: int = 15):
        super().__init__()
        if not hasattr(state, 'proof_complete'):
            raise TypeError("state doit avoir proof_complete")
        self._state = state
        self._max_iterations = max_iterations
        self._logger = logging.getLogger("ProofTerminationStrategy")

    async def should_terminate(self, agent: Any, history: List[ChatMessageContent]) -> bool:
        """Verifie si la conversation doit se terminer."""
        # 1. Preuve trouvee
        try:
            if self._state.proof_complete:
                self._logger.info("Terminaison: preuve complete")
                return True
        except Exception as e:
            self._logger.error(f"Erreur verification preuve: {e}")

        # 2. Max iterations
        try:
            if self._state.iteration >= self._max_iterations:
                self._logger.info(f"Terminaison: max iterations ({self._max_iterations})")
                return True
        except Exception as e:
            self._logger.error(f"Erreur verification iterations: {e}")

        # 3. Phase complete ou failed
        try:
            if self._state.phase.value in ["complete", "failed"]:
                self._logger.info(f"Terminaison: phase {self._state.phase.value}")
                return True
        except:
            pass

        return False

    async def reset(self) -> None:
        pass


# =============================================================================
# ProofAgentGroupChat - Orchestration multi-agents
# =============================================================================

class ProofAgentGroupChat:
    """
    Orchestre les agents pour la preuve de theoremes.
    Utilise ProofSelectionStrategy et ProofTerminationStrategy.
    """

    def __init__(self, agents: Dict[str, Any], state: ProofState, use_sk: bool = True):
        self.agents = agents
        self.state = state
        self.use_sk = use_sk and SK_AVAILABLE
        self.history = []

    def run(self, initial_message: str, verbose: bool = True) -> str:
        """Execute la conversation multi-agents."""
        if self.use_sk:
            return asyncio.get_event_loop().run_until_complete(
                self._run_sk(initial_message, verbose)
            )
        else:
            return self._run_fallback(initial_message, verbose)

    async def _run_sk(self, initial_message: str, verbose: bool) -> str:
        """Execution avec Semantic Kernel et strategies personnalisees."""
        if verbose:
            print("=" * 60)
            print(f"Session SK demarree: {initial_message[:80]}...")
            print("=" * 60)

        agent_list = list(self.agents.values())

        if verbose:
            print(f"[LOG] Agents: {[a.name for a in agent_list]}")
            print(f"[LOG] Max iterations: {self.state.max_iterations}")

        # Creer les strategies basees sur l'etat partage
        selection_strategy = ProofSelectionStrategy(
            agents=agent_list,
            state=self.state,
            default_agent_name="SearchAgent"
        )

        termination_strategy = ProofTerminationStrategy(
            state=self.state,
            max_iterations=self.state.max_iterations
        )

        if verbose:
            print(f"[LOG] Strategies initialisees (basees sur etat partage)")

        # Creer le chat SK
        chat = AgentGroupChat(
            agents=agent_list,
            selection_strategy=selection_strategy,
            termination_strategy=termination_strategy,
        )

        await chat.add_chat_message(message=initial_message)

        iteration = 0
        if verbose:
            print(f"[LOG] Demarrage boucle multi-agents...")

        try:
            async for response in chat.invoke():
                iteration += 1
                self.state.iteration = iteration

                if response is None:
                    if verbose:
                        print(f"[Tour {iteration}] Response None, continue...")
                    continue

                agent_name = getattr(response, 'name', None) or 'Unknown'
                content = str(response.content)[:500] if response.content else "(vide)"

                if verbose:
                    print(f"\n[Tour {iteration}/{self.state.max_iterations}] Agent: {agent_name}")
                    print(f"  Phase: {self.state.phase.value}")
                    print(f"  Response: {content[:300]}{'...' if len(content) > 300 else ''}")

                self.history.append({
                    "agent": agent_name,
                    "response": content,
                    "iteration": iteration,
                    "phase": self.state.phase.value,
                })

                if self.state.proof_complete:
                    if verbose:
                        print(f"[LOG] Preuve complete detectee!")
                    break

        except Exception as e:
            if verbose:
                print(f"\n[ERREUR Tour {iteration}] {type(e).__name__}: {e}")
                import traceback
                traceback.print_exc()

        if verbose:
            print("\n" + "=" * 60)
            print(f"SESSION TERMINEE - {iteration} tours executes")
            if self.state.proof_complete:
                print("SUCCES! Preuve trouvee:")
                print(self.state.final_proof)
            else:
                print(f"Pas de preuve trouvee.")
                if self.state.last_error:
                    print(f"Derniere erreur: {self.state.last_error}")
            print("=" * 60)

        return self.state.final_proof or "Preuve non trouvee"

    def _run_fallback(self, initial_message: str, verbose: bool = True) -> str:
        """Execution sans Semantic Kernel (mode simulation)."""
        if verbose:
            print("=" * 60)
            print(f"Session FALLBACK: {initial_message[:80]}...")
            print("=" * 60)

        current_message = initial_message
        agent_order = ["SearchAgent", "TacticAgent", "VerifierAgent", "CriticAgent", "CoordinatorAgent"]

        for i in range(self.state.max_iterations):
            designated = self.state.consume_next_agent_designation()
            if designated and designated in self.agents:
                agent_name = designated
            else:
                agent_name = agent_order[i % len(agent_order)]

            agent = self.agents.get(agent_name)
            if not agent:
                continue

            if verbose:
                print(f"\n[Tour {self.state.iteration_count + 1}] {agent_name}")

            response = agent.invoke(current_message, self.state)

            self.history.append({
                "iteration": self.state.iteration_count,
                "agent": agent_name,
                "response": response[:200],
            })

            if verbose:
                print(f"  Response: {response[:200]}...")

            if self.state.proof_complete:
                if verbose:
                    print(f"\n[LOG] Preuve trouvee!")
                break

            current_message = response

        if verbose:
            print("\n" + "=" * 60)
            print(f"Session terminee apres {self.state.iteration_count} iterations.")
            print("=" * 60)

        return self.state.final_proof or "Preuve non trouvee"


# =============================================================================
# Test des Strategies
# =============================================================================

print("=== Test des Strategies ===")

test_state = ProofState(
    theorem_statement="theorem test (n : Nat) : n = n",
    current_goal="n = n",
    max_iterations=5
)

print(f"State cree: {test_state.session_id}")
print(f"Phase initiale: {test_state.phase.value}")

# Test designation
test_state.designate_next_agent("TacticAgent")
designated = test_state.consume_next_agent_designation()
print(f"Designation test: {designated}")

# Test proof_complete
print(f"proof_complete initial: {test_state.proof_complete}")
test_state.phase = ProofPhase.COMPLETE
print(f"proof_complete apres COMPLETE: {test_state.proof_complete}")

print("\nStrategies pretes pour utilisation avec AgentGroupChat")


=== Test des Strategies ===
State cree: a3882044
Phase initiale: init
Designation test: TacticAgent
proof_complete initial: False
proof_complete apres COMPLETE: True

Strategies pretes pour utilisation avec AgentGroupChat


## 🧪 Démonstrations Progressives

### Objectif

Valider que le système multi-agents **fonctionne réellement** sur des problèmes de complexité croissante.

### Les 3 démos

#### 1️⃣ DEMO_1_TRIVIAL : `theorem demo_rfl (n : Nat) : n = n`

- **Complexité** : Triviale (égalité réflexive)
- **Preuve attendue** : `by rfl` (une seule tactique)
- **Itérations attendues** : 1-2
- **Lemmes nécessaires** : 0 (tautologie)
- **But** : Vérifier que le système peut résoudre le cas le plus simple

#### 2️⃣ DEMO_2_SIMPLE : `theorem nat_add_zero (n : Nat) : n + 0 = n`

- **Complexité** : Simple (propriété arithmétique basique)
- **Preuve attendue** : `by rw [Nat.add_zero]` ou induction
- **Itérations attendues** : 4-6
- **Lemmes nécessaires** : 1-2 (de Mathlib)
- **But** : Tester **SearchAgent** (recherche de lemmes) + **TacticAgent**

#### 3️⃣ DEMO_3_INTERMEDIATE : `theorem nat_add_comm (n m : Nat) : n + m = m + n`

- **Complexité** : Intermédiaire (commutativité de l'addition)
- **Preuve attendue** : Induction + réécriture avec plusieurs lemmes
- **Itérations attendues** : 8-12
- **Lemmes nécessaires** : 2-3 (Nat.add_comm, Nat.add_succ, etc.)
- **But** : Tester **orchestration complète** avec backtracking potentiel

### Métriques à comparer

| Métrique | Démo 1 | Démo 2 | Démo 3 |
|----------|--------|--------|--------|
| Itérations | 1-2 | 4-6 | 8-12 |
| Lemmes découverts | 0 | 1-2 | 2-3 |
| Tactiques essayées | 1 | 2-3 | 4-6 |
| Vérifications Lean | 1 | 1-2 | 2-3 |

### Hypothèse à valider

**Le système multi-agents SCALE avec la complexité du problème.**

Si Démo 3 prend ~6× plus d'itérations que Démo 1, c'est **normal et attendu** (pas un bug).

Si Démo 3 échoue alors que Démo 1 réussit, ça indique un problème d'orchestration.

In [ ]:
# =============================================================================
# Section 8.8 - Demonstration Complete
# =============================================================================

def prove_with_multi_agents(
    theorem: str,
    goal: str = "",
    max_iterations: int = 20,
    verbose: bool = True,
    use_simulation: bool = None  # None = auto-detect
) -> Dict[str, Any]:
    """
    Prouve un theoreme en utilisant le systeme multi-agents.

    Args:
        theorem: L'enonce du theoreme complet
        goal: Le but a prouver (extrait du theoreme si non fourni)
        max_iterations: Nombre maximum d'iterations
        verbose: Afficher les logs
        use_simulation: True=simulation, False=LLM reel, None=auto

    Returns:
        Dict avec resultats et metriques
    """
    import time
    start_time = time.time()

    # Auto-detection du mode
    if use_simulation is None:
        api_key = os.getenv("OPENAI_API_KEY", "")
        has_valid_key = api_key and len(api_key) > 10 and not api_key.startswith("sk-...")
        use_simulation = not has_valid_key

    # 1. Creer l'etat
    if not goal:
        if ":" in theorem:
            goal = theorem.split(":")[-1].strip()

    state = ProofState(
        theorem_statement=theorem,
        current_goal=goal,
        max_iterations=max_iterations
    )

    # 2. Creer le runner Lean
    runner = LeanRunner(backend="subprocess", timeout=30)

    # 3. Creer les plugins
    plugins = {
        "state": ProofStateManagerPlugin(state),
        "search": LeanSearchPlugin(runner),
        "tactic": LeanTacticPlugin(),
        "verification": LeanVerificationPlugin(runner)
    }

    # 4. Creer les agents
    use_sk = SK_AVAILABLE and not use_simulation
    agents = create_agents(plugins, state, use_sk=use_sk, use_simulation=use_simulation)

    # 5. Configurer les strategies
    # Strategies gerees automatiquement par ProofAgentGroupChat

    # 6. Creer le groupe de chat
    chat = ProofAgentGroupChat(
        agents=agents,
        state=state,
        use_sk=use_sk
    )

    mode_str = "Semantic Kernel" if use_sk else ("Simulation" if use_simulation else "OpenAI direct")
    if verbose:
        print(f"Mode: {mode_str}")

    # 7. Executer
    result = chat.run(f"Prouver: {theorem}", verbose=verbose)

    # 8. Collecter les metriques
    elapsed = time.time() - start_time
    metrics = {
        "success": state.proof_complete,
        "theorem": theorem,
        "final_proof": state.final_proof,
        "iterations": state.iteration_count,
        "lemmas_discovered": len(state.discovered_lemmas),
        "tactics_tried": len(state.tactics_history),
        "verifications": len(state.verification_results),
        "total_time_s": round(elapsed, 2),
        "lean_time_ms": round(state.total_lean_time_ms, 2),
        "mode": mode_str
    }

    return metrics


# =============================================================================
# Test de la demonstration
# =============================================================================

print("\n" + "=" * 60)
print("DEMONSTRATION MULTI-AGENTS POUR THEOREM PROVING")
print("=" * 60)

# =============================================================================
# Section 8.8 - Démonstrations Progressives Multi-Agents
# =============================================================================

# Configuration
USE_LLM_MODE = True  # True pour LLM réel, False pour simulation

# Quatre théorèmes de complexité croissante
DEMOS = [
    {
        "name": "DEMO_1_TRIVIAL",
        "theorem": "theorem demo_rfl (n : Nat) : n = n",
        "description": "Égalité réflexive (1-2 itérations attendues)",
        "expected_iterations": "1-2",
        "expected_lemmas": "0",
        "complexity": "Triviale - teste rfl uniquement"
    },
    {
        "name": "DEMO_2_SIMPLE",
        "theorem": "theorem add_right_cancel (a b c : Nat) : a + b = c + b -> a = c",
        "description": "Simplification addition (6-10 itérations attendues)",
        "expected_iterations": "6-10",
        "expected_lemmas": "2-3",
        "complexity": "Simple - necessite Nat.add_right_cancel ou decomposition"
    },
    {
        "name": "DEMO_3_INTERMEDIATE",
        "theorem": "theorem mul_add_distr (a b c : Nat) : a * (b + c) = a * b + a * c",
        "description": "Distributivité multiplication (10-15 itérations attendues)",
        "expected_iterations": "10-15",
        "expected_lemmas": "3-5",
        "complexity": "Intermédiaire - composition Nat.mul_add + associativite"
    },
    {
        "name": "DEMO_4_ADVANCED",
        "theorem": "theorem list_length_append (l1 l2 : List Nat) : (l1 ++ l2).length = l1.length + l2.length",
        "description": "Induction sur listes (12-20 itérations attendues)",
        "expected_iterations": "12-20",
        "expected_lemmas": "4-6",
        "complexity": "Avance - induction structurelle, trigger CriticAgent"
    }
]

print("=" * 70)
print("DÉMONSTRATIONS PROGRESSIVES - SYSTÈME MULTI-AGENTS")
print("=" * 70)
print()

# Exécuter chaque démo
results_comparison = []



### Execution DEMO_1 : Preuve Triviale

**Objectif** : Valider le pipeline complet avec un theoreme trivial

**Theoreme** : `theorem demo_rfl (n : Nat) : n = n`

**Attentes** :
- **Iterations** : 1-2 (reflexivite immediate)
- **Agents impliques** : TacticAgent (rfl) → VerifierAgent
- **CriticAgent/CoordinatorAgent** : NON (preuve triviale)
- **Temps** : <1 seconde

Cette demo sert de **baseline** pour verifier que le systeme fonctionne.


In [ ]:
# Execute DEMO_1demo = DEMOS[0]print("\n" + "=" * 70)print(f"DEMO 1/4: {demo['name']}")print("=" * 70)print(f"Theoreme: {demo['theorem']}")print(f"Complexite: {demo['complexity']}")print(f"Iterations attendues: {demo['expected_iterations']}")print(f"Lemmes necessaires: {demo['expected_lemmas']}")print("=" * 70)result_1 = prove_with_multi_agents(    theorem=demo["theorem"],    max_iterations=20,    verbose=True,    use_simulation=not USE_LLM_MODE)print(f"\nResultat DEMO_1:")print(f"  - Success: {result_1['success']}")print(f"  - Iterations: {result_1['iterations']}")print(f"  - Proof: {result_1['proof'][:100] if result_1['proof'] else 'None'}...")

### Execution DEMO_2 : Preuve Simple

**Objectif** : Tester recherche de lemmes + composition

**Theoreme** : `theorem add_right_cancel (a b c : Nat) : a + b = c + b -> a = c`

**Attentes** :
- **Iterations** : 6-10 (recherche lemme + application)
- **Agents impliques** : SearchAgent → TacticAgent → VerifierAgent
- **Lemmes Mathlib attendus** : `Nat.add_right_cancel`, `Nat.add_comm`
- **CriticAgent/CoordinatorAgent** : POSSIBLE si lemme pas trouve directement
- **Temps** : 2-5 secondes

Cette demo teste la **recherche de lemmes** et la **generation de tactiques** adaptees.


In [ ]:
# Execute DEMO_2demo = DEMOS[1]print("\n" + "=" * 70)print(f"DEMO 2/4: {demo['name']}")print("=" * 70)print(f"Theoreme: {demo['theorem']}")print(f"Complexite: {demo['complexity']}")print(f"Iterations attendues: {demo['expected_iterations']}")print(f"Lemmes necessaires: {demo['expected_lemmas']}")print("=" * 70)result_2 = prove_with_multi_agents(    theorem=demo["theorem"],    max_iterations=20,    verbose=True,    use_simulation=not USE_LLM_MODE)print(f"\nResultat DEMO_2:")print(f"  - Success: {result_2['success']}")print(f"  - Iterations: {result_2['iterations']}")print(f"  - Proof: {result_2['proof'][:100] if result_2['proof'] else 'None'}...")

### Execution DEMO_3 : Preuve Intermediaire

**Objectif** : Tester composition de plusieurs lemmes

**Theoreme** : `theorem mul_add_distr (a b c : Nat) : a * (b + c) = a * b + a * c`

**Attentes** :
- **Iterations** : 10-15 (composition lemmes)
- **Agents impliques** : SearchAgent (multiple) → TacticAgent → VerifierAgent → CriticAgent (si echec)
- **Lemmes Mathlib attendus** : `Nat.mul_add`, `Nat.mul_comm`, `Nat.add_assoc`
- **CriticAgent** : PROBABLE (necessite ajustements tactiques)
- **Temps** : 5-10 secondes

Cette demo teste l'**orchestration multi-agents** avec feedback loops.


In [ ]:
# Execute DEMO_3demo = DEMOS[2]print("\n" + "=" * 70)print(f"DEMO 3/4: {demo['name']}")print("=" * 70)print(f"Theoreme: {demo['theorem']}")print(f"Complexite: {demo['complexity']}")print(f"Iterations attendues: {demo['expected_iterations']}")print(f"Lemmes necessaires: {demo['expected_lemmas']}")print("=" * 70)result_3 = prove_with_multi_agents(    theorem=demo["theorem"],    max_iterations=20,    verbose=True,    use_simulation=not USE_LLM_MODE)print(f"\nResultat DEMO_3:")print(f"  - Success: {result_3['success']}")print(f"  - Iterations: {result_3['iterations']}")print(f"  - Proof: {result_3['proof'][:100] if result_3['proof'] else 'None'}...")

### Execution DEMO_4 : Preuve Avancee

**Objectif** : Stresser le systeme avec induction structurelle

**Theoreme** : `theorem list_length_append (l1 l2 : List Nat) : (l1 ++ l2).length = l1.length + l2.length`

**Attentes** :
- **Iterations** : 12-20 (induction + lemmes auxiliaires)
- **Agents impliques** : SearchAgent → TacticAgent (induction) → VerifierAgent → **CriticAgent** → CoordinatorAgent (si blocage)
- **Lemmes Mathlib attendus** : `List.length_append`, `List.length_cons`, `Nat.succ_add`
- **Strategies** : EXPLORATION → REFINEMENT → VALIDATION
- **CriticAgent/CoordinatorAgent** : **REQUIS** (echecs de tactiques attendus)
- **Temps** : 10-30 secondes

Cette demo doit **declencher CriticAgent** si la tactique d'induction echoue ou si les lemmes ne suffisent pas. C'est le seul theoreme qui devrait stresser l'orchestration complete.

**Note** : Si DEMO_4 se complete en <10 iterations sans CriticAgent, cela signifie que Mathlib contient le lemme directement et le theoreme n'est pas assez complexe.


In [ ]:
# Execute DEMO_4demo = DEMOS[3]print("\n" + "=" * 70)print(f"DEMO 4/4: {demo['name']}")print("=" * 70)print(f"Theoreme: {demo['theorem']}")print(f"Complexite: {demo['complexity']}")print(f"Iterations attendues: {demo['expected_iterations']}")print(f"Lemmes necessaires: {demo['expected_lemmas']}")print("=" * 70)result_4 = prove_with_multi_agents(    theorem=demo["theorem"],    max_iterations=20,    verbose=True,    use_simulation=not USE_LLM_MODE)print(f"\nResultat DEMO_4:")print(f"  - Success: {result_4['success']}")print(f"  - Iterations: {result_4['iterations']}")print(f"  - Proof: {result_4['proof'][:100] if result_4['proof'] else 'None'}...")

## 🎼 Harmonic Aristotle : Décomposition Récursive

### Contexte

**Technique développée par DeepSeek (2024)** pour résoudre des problèmes de théorie des nombres ouverts depuis 30+ ans.

### Le problème des preuves "monolithiques"

Approche classique (linéaire) :

```
Théorème T : n + m = m + n
  ↓
Recherche de lemmes
  ↓
Génération de tactiques
  ↓
Vérification
  ↓
Succès ou échec
```

**Problème** : Si le théorème est complexe, la recherche de lemmes devient explosive (trop de candidats).

### Idée centrale : Décomposition récursive

Au lieu de prouver T directement, **décomposer T en sous-théorèmes plus simples** :

```
Théorème T : n + m = m + n
  ↓ DÉCOMPOSITION
  ├─ T1 : n + 0 = 0 + n (plus facile)
  ├─ T2 : n + (m + 1) = (m + 1) + n (plus facile)
  └─ T3 : Induction utilisant T1 et T2 (maintenant facile!)
```

### Exemple concret

**Sans décomposition** :

```lean
theorem add_comm (n m : Nat) : n + m = m + n := by
  -- Recherche de lemmes : 50+ candidats dans Mathlib
  -- Génération de tactiques : Quelle induction ? Sur n ou m ?
  -- Vérifications : 10-15 tentatives
  -- ❌ Complexité explosive
```

**Avec décomposition (Harmonic Aristotle)** :

```lean
-- Étape 1 : Prouver cas de base
theorem add_zero (n : Nat) : n + 0 = n := by rfl

-- Étape 2 : Prouver cas successeur
theorem add_succ (n m : Nat) : n + (m + 1) = (n + m) + 1 := by rfl

-- Étape 3 : Combiner pour prouver commutativité (facile maintenant!)
theorem add_comm (n m : Nat) : n + m = m + n := by
  induction m with
  | zero => rw [add_zero, zero_add]  -- Utilise add_zero
  | succ m ih => rw [add_succ, ih, succ_add]  -- Utilise add_succ
```

### Métrique clé : **Réduction de l'espace de recherche**

| Approche | Lemmes candidats | Tactiques essayées | Succès |
|----------|------------------|-------------------|--------|
| Linéaire | 50+ | 15-20 | 40% |
| Harmonic Aristotle | 5-10 (par sous-théorème) | 5-8 (total) | 85% |

### Intégration dans notre système

Harmonic Aristotle s'intègre comme **stratégie de CriticAgent** :

1. CriticAgent détecte que le théorème est complexe (>5 itérations sans succès)
2. Propose une décomposition en sous-théorèmes
3. CoordinatorAgent orchestre la preuve des sous-théorèmes
4. TacticAgent combine les résultats

**Résultat** : Résolution de problèmes ouverts (Erdos #124 variant en 6h).

## 6. Techniques de Harmonic Aristotle

### 6.1 Decomposition de problemes

Aristotle decompose les problemes complexes en sous-problemes plus simples.

In [109]:
class AristotleDecomposer:
    """
    Decomposition de problemes a la Harmonic Aristotle.
    """
    
    def decompose(self, theorem: str) -> List[str]:
        """
        Decompose un theoreme en sous-lemmes.
        
        Strategy:
        1. Identifier la structure (conjonction, equivalence, etc.)
        2. Separer en composantes
        3. Identifier les dependances
        """
        subproblems = []
        
        # Decomposition basique par structure
        if "<->" in theorem or "iff" in theorem.lower():
            # Equivalence = deux implications
            parts = theorem.split("<->")
            subproblems.append(f"Direction 1: {parts[0]} -> {parts[1]}")
            subproblems.append(f"Direction 2: {parts[1]} -> {parts[0]}")
        
        elif "/\\" in theorem or "and" in theorem.lower():
            # Conjonction = prouver chaque partie
            parts = theorem.split("/\\")
            for i, part in enumerate(parts):
                subproblems.append(f"Partie {i+1}: {part.strip()}")
        
        elif "forall" in theorem.lower():
            # Universel = fixer variable, prouver pour arbitraire
            subproblems.append(f"Generalisation: introduire variable, prouver corps")
        
        elif "exists" in theorem.lower():
            # Existentiel = trouver temoin + preuve
            subproblems.append(f"Temoin: trouver valeur concrete")
            subproblems.append(f"Verification: prouver pour ce temoin")
        
        else:
            # Pas de decomposition evidente
            subproblems.append(theorem)
        
        return subproblems
    
    def solve_hierarchical(self, theorem: str, solver) -> Tuple[bool, str]:
        """
        Resolution hierarchique par decomposition.
        """
        subproblems = self.decompose(theorem)
        
        if len(subproblems) == 1 and subproblems[0] == theorem:
            # Cas de base: resoudre directement
            return solver(theorem)
        
        # Resoudre chaque sous-probleme
        solutions = []
        for sub in subproblems:
            success, proof = self.solve_hierarchical(sub, solver)
            if not success:
                return False, None
            solutions.append(proof)
        
        # Combiner les solutions
        combined = self._combine_proofs(solutions)
        return True, combined
    
    def _combine_proofs(self, proofs: List[str]) -> str:
        """Combine des preuves de sous-problemes."""
        return "\n".join([
            f"-- Partie {i+1}\n{proof}" 
            for i, proof in enumerate(proofs)
        ])

# Test
decomposer = AristotleDecomposer()
subproblems = decomposer.decompose("P <-> Q")
print("Decomposition de 'P <-> Q':")
for sp in subproblems:
    print(f"  - {sp}")

Decomposition de 'P <-> Q':
  - Direction 1: P  ->  Q
  - Direction 2:  Q -> P 


## 7. Benchmarking sur Problemes d'Erdos

Les problemes d'Erdos sont devenus le benchmark de reference pour evaluer les systemes de theorem proving automatique. Plusieurs ont ete resolus par IA en 2025-2026.

In [110]:
# Benchmark sur des problemes type Erdos (simplifies)

BENCHMARK_PROBLEMS = [
    {
        "id": "simple_1",
        "name": "Addition zero",
        "statement": "theorem add_zero (n : Nat) : n + 0 = n",
        "difficulty": 1,
        "expected_tactics": ["exact Nat.add_zero n", "rfl"]
    },
    {
        "id": "simple_2", 
        "name": "Commutativite addition",
        "statement": "theorem add_comm (a b : Nat) : a + b = b + a",
        "difficulty": 2,
        "expected_tactics": ["exact Nat.add_comm a b"]
    },
    {
        "id": "medium_1",
        "name": "Associativite addition",
        "statement": "theorem add_assoc (a b c : Nat) : (a + b) + c = a + (b + c)",
        "difficulty": 3,
        "expected_tactics": ["exact Nat.add_assoc a b c", "induction c"]
    },
]

def run_benchmark(solver, problems=BENCHMARK_PROBLEMS):
    """Execute le benchmark sur les problemes donnes."""
    results = []
    
    for problem in problems:
        print(f"\nTest: {problem['name']} (difficulte: {problem['difficulty']})")
        
        success, proof = solver.prove(problem['statement'])
        
        results.append({
            "id": problem["id"],
            "success": success,
            "proof": proof
        })
    
    # Statistiques
    total = len(results)
    solved = sum(1 for r in results if r["success"])
    
    print(f"\n{'='*60}")
    print(f"RESULTATS DU BENCHMARK")
    print(f"{'='*60}")
    print(f"Resolus: {solved}/{total} ({100*solved/total:.1f}%)")
    
    return results

# Executer le benchmark (limite a 3 iterations pour la demo)
orchestrator.max_iterations = 3
results = run_benchmark(orchestrator, BENCHMARK_PROBLEMS[:2])


Test: Addition zero (difficulte: 1)

Debut de la preuve: theorem add_zero (n : Nat) : n + 0 = n

--- Iteration 1 ---
Lemmes trouves: ['Nat.add_zero', 'Nat.zero_add', 'Nat.add_comm']
Tactiques generees: ['rfl']

Preuve trouvee!

Test: Commutativite addition (difficulte: 2)

Debut de la preuve: theorem add_comm (a b : Nat) : a + b = b + a

--- Iteration 1 ---
Lemmes trouves: ['Nat.add_zero', 'Nat.zero_add', 'Nat.add_comm']
Tactiques generees: ['rfl']

Preuve trouvee!

RESULTATS DU BENCHMARK
Resolus: 2/2 (100.0%)


## 8. Exercices

### Exercice 1 : Ameliorer l'agent de recherche

In [111]:
# Exercice 1 - SOLUTION: Agent de recherche ameliore avec scoring LLM

import os
import sys
from pathlib import Path

# Ajouter le repertoire courant au path
sys.path.insert(0, str(Path.cwd()))

# Utiliser load_env_file de lean_runner (evite les problemes d'introspection)
from lean_runner import load_env_file
env_path = Path.cwd() / ".env"
load_env_file(env_path)

class ImprovedSearchAgent(TheoremSearchAgent):
    """
    Version amelioree de l'agent de recherche avec scoring par LLM.
    
    Ameliorations:
    1. Scoring semantique par LLM (pertinence reelle, pas juste mots-cles)
    2. Cache des scores pour eviter les appels API redondants
    3. Fallback sur heuristique si API non disponible
    """
    
    def __init__(self, llm_client=None):
        super().__init__(llm_client)
        self.score_cache = {}  # (lemma_name, goal) -> score
        self.api_available = self._check_api()
    
    def _check_api(self) -> bool:
        """Verifie si l'API OpenAI est disponible."""
        api_key = os.getenv("OPENAI_API_KEY")
        return api_key is not None and not api_key.startswith("sk-...")
    
    def _score_with_llm(self, lemma: Lemma, goal: str) -> float:
        """
        Score la pertinence d'un lemme par rapport au but en utilisant un LLM.
        
        Returns:
            Score de pertinence entre 0.0 et 1.0
        """
        # Verifier le cache
        cache_key = (lemma.name, goal)
        if cache_key in self.score_cache:
            return self.score_cache[cache_key]
        
        # Si API non disponible, utiliser heuristique
        if not self.api_available:
            score = self._heuristic_score(lemma, goal)
            self.score_cache[cache_key] = score
            return score
        
        # Appel API reel
        try:
            from openai import OpenAI
            client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
            
            prompt = f"""Evalue la pertinence d'un lemme mathematique pour prouver un but en Lean 4.

Lemme: {lemma.name}
Enonce du lemme: {lemma.statement}

But a prouver: {goal}

Sur une echelle de 0 a 1, quelle est la pertinence de ce lemme?
- 1.0 = Le lemme resout directement le but
- 0.7-0.9 = Tres pertinent, peut etre utilise avec une reecriture
- 0.4-0.6 = Moderement pertinent, structure similaire
- 0.1-0.3 = Peu pertinent, meme domaine mais different
- 0.0 = Aucun rapport

Reponds UNIQUEMENT avec un nombre decimal entre 0 et 1."""

            # Les modeles modernes (gpt-4o, gpt-4.5, gpt-5, o1, o3) utilisent max_completion_tokens
            model = os.getenv("OPENAI_CHAT_MODEL_ID", "gpt-5.2")
            use_max_completion_tokens = any(model.startswith(p) for p in ('gpt-4o', 'gpt-4.5', 'gpt-5', 'o1', 'o3'))
            token_param = {"max_completion_tokens": 10} if use_max_completion_tokens else {"max_tokens": 10}
            
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.1,
                **token_param
            )
            
            # Parser la reponse
            score_text = response.choices[0].message.content.strip()
            score = float(score_text)
            score = max(0.0, min(1.0, score))  # Clamp entre 0 et 1
            
        except Exception as e:
            print(f"  [Scoring LLM echoue: {e}, utilisation heuristique]")
            score = self._heuristic_score(lemma, goal)
        
        # Mettre en cache
        self.score_cache[cache_key] = score
        return score
    
    def _heuristic_score(self, lemma: Lemma, goal: str) -> float:
        """
        Score heuristique base sur la correspondance de termes.
        Utilise comme fallback quand l'API n'est pas disponible.
        """
        # Normaliser les chaines
        lemma_terms = set(lemma.statement.lower().replace(":", " ").split())
        goal_terms = set(goal.lower().replace(":", " ").split())
        
        # Score = Jaccard similarity
        intersection = len(lemma_terms & goal_terms)
        union = len(lemma_terms | goal_terms)
        
        if union == 0:
            return 0.0
        
        jaccard = intersection / union
        
        # Bonus si le nom du lemme correspond au type d'operation
        bonus = 0.0
        if "add" in lemma.name.lower() and "+" in goal:
            bonus = 0.2
        elif "mul" in lemma.name.lower() and "*" in goal:
            bonus = 0.2
        elif "comm" in lemma.name.lower() and ("comm" in goal.lower() or 
                                               ("+b" in goal.replace(" ", "") and "+a" in goal.replace(" ", ""))):
            bonus = 0.15
        
        return min(1.0, jaccard + bonus)
    
    def _score_lemmas(self, lemmas: List[Lemma], goal: str) -> List[Lemma]:
        """Score les lemmes avec la methode amelioree."""
        print(f"  Scoring {len(lemmas)} lemmes...")
        
        for lemma in lemmas:
            lemma.relevance_score = self._score_with_llm(lemma, goal)
        
        # Trier par pertinence decroissante
        return sorted(lemmas, key=lambda l: l.relevance_score, reverse=True)

# Test de l'agent ameliore
print("Test de ImprovedSearchAgent:")
print("-" * 40)

improved_agent = ImprovedSearchAgent()
goal = "n + 0 = n"
results = improved_agent.search(goal)

print(f"\nLemmes trouves pour '{goal}':")
for lemma in results:
    print(f"  [{lemma.relevance_score:.2f}] {lemma.name}: {lemma.statement}")

# Test sur un autre but
goal2 = "a + b = b + a"
results2 = improved_agent.search(goal2)
print(f"\nLemmes trouves pour '{goal2}':")
for lemma in results2:
    print(f"  [{lemma.relevance_score:.2f}] {lemma.name}: {lemma.statement}")

Test de ImprovedSearchAgent:
----------------------------------------
  Scoring 8 lemmes...

Lemmes trouves pour 'n + 0 = n':
  [1.00] Nat.add_zero: n + 0 = n
  [0.50] Nat.zero_add: 0 + n = n
  [0.50] Nat.add_comm: n + m = m + n
  [0.20] Nat.add_assoc: (n + m) + k = n + (m + k)
  [0.20] Nat.succ_add: succ n + m = succ (n + m)
  [0.20] Nat.add_succ: n + succ m = succ (n + m)
  [0.10] Nat.mul_zero: n * 0 = 0
  [0.00] Nat.zero_mul: 0 * n = 0
  Scoring 6 lemmes...

Lemmes trouves pour 'a + b = b + a':
  [1.00] Nat.add_comm: n + m = m + n
  [0.20] Nat.add_zero: n + 0 = n
  [0.20] Nat.zero_add: 0 + n = n
  [0.20] Nat.add_assoc: (n + m) + k = n + (m + k)
  [0.20] Nat.succ_add: succ n + m = succ (n + m)
  [0.20] Nat.add_succ: n + succ m = succ (n + m)


### Exercice 2 : Ajouter de la memoire

In [112]:
# Exercice 2 - SOLUTION: Systeme de memoire avec pattern matching

import re
import json
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass, field
from difflib import SequenceMatcher

@dataclass
class StoredProof:
    """Une preuve stockee avec son contexte."""
    theorem_pattern: str
    original_theorem: str
    proof: str
    success_count: int = 1
    variables: Dict[str, str] = field(default_factory=dict)

class ProofMemory:
    """
    Systeme de memoire pour reutiliser les preuves reussies.
    
    Fonctionnalites:
    1. Pattern matching pour generaliser les theoremes
    2. Recherche de preuves similaires par similarite
    3. Adaptation des preuves au nouveau contexte
    4. Persistence (optionnelle) vers fichier JSON
    """
    
    def __init__(self, similarity_threshold: float = 0.7):
        self.proofs: Dict[str, StoredProof] = {}  # pattern -> StoredProof
        self.similarity_threshold = similarity_threshold
    
    def store(self, theorem: str, proof: str) -> str:
        """
        Stocke une preuve reussie.
        
        Returns:
            L'ID du pattern utilise pour le stockage
        """
        # Extraire le pattern et les variables
        pattern, variables = self._extract_pattern(theorem)
        
        if pattern in self.proofs:
            # Incrementer le compteur de succes
            self.proofs[pattern].success_count += 1
        else:
            # Nouvelle preuve
            self.proofs[pattern] = StoredProof(
                theorem_pattern=pattern,
                original_theorem=theorem,
                proof=proof,
                variables=variables
            )
        
        return pattern
    
    def recall(self, theorem: str) -> Optional[Tuple[str, float]]:
        """
        Retrouve une preuve similaire.
        
        Returns:
            (preuve_adaptee, score_similarite) ou None si rien trouve
        """
        # Extraire le pattern du theoreme
        query_pattern, query_vars = self._extract_pattern(theorem)
        
        # Recherche exacte d'abord
        if query_pattern in self.proofs:
            stored = self.proofs[query_pattern]
            adapted_proof = self._adapt_proof(stored.proof, stored.variables, query_vars)
            return adapted_proof, 1.0
        
        # Recherche par similarite
        best_match = None
        best_score = 0.0
        
        for pattern, stored in self.proofs.items():
            score = self._similarity(query_pattern, pattern)
            if score > best_score and score >= self.similarity_threshold:
                best_score = score
                best_match = stored
        
        if best_match:
            adapted_proof = self._adapt_proof(best_match.proof, best_match.variables, query_vars)
            return adapted_proof, best_score
        
        return None
    
    def _extract_pattern(self, theorem: str) -> Tuple[str, Dict[str, str]]:
        """
        Extrait un pattern generalise du theoreme.
        
        Transformations:
        - Variables specifiques -> placeholders (?x, ?y, ?z)
        - Types conserves
        - Structure preservee
        
        Exemple:
            "theorem foo (n : Nat) : n + 0 = n" 
            -> "theorem ?name (?x : Nat) : ?x + 0 = ?x"
        """
        variables = {}
        pattern = theorem
        
        # Extraire le nom du theoreme
        name_match = re.search(r'theorem\s+(\w+)', theorem)
        if name_match:
            variables['theorem_name'] = name_match.group(1)
            pattern = re.sub(r'theorem\s+\w+', 'theorem ?name', pattern)
        
        # Extraire les variables de type Nat/Int
        var_matches = re.findall(r'\((\w+)\s*:\s*(\w+)\)', theorem)
        placeholder_index = 0
        placeholders = ['?x', '?y', '?z', '?a', '?b', '?c']
        
        for var_name, var_type in var_matches:
            if placeholder_index < len(placeholders):
                placeholder = placeholders[placeholder_index]
                variables[placeholder] = var_name
                # Remplacer la variable dans tout le pattern
                pattern = re.sub(rf'\b{var_name}\b', placeholder, pattern)
                placeholder_index += 1
        
        return pattern, variables
    
    def _similarity(self, pattern1: str, pattern2: str) -> float:
        """
        Calcule la similarite entre deux patterns.
        Utilise SequenceMatcher pour une comparaison robuste.
        """
        # Normaliser
        p1 = pattern1.lower().replace(" ", "")
        p2 = pattern2.lower().replace(" ", "")
        
        return SequenceMatcher(None, p1, p2).ratio()
    
    def _adapt_proof(self, proof: str, original_vars: Dict[str, str], 
                     new_vars: Dict[str, str]) -> str:
        """
        Adapte une preuve au nouveau contexte en substituant les variables.
        """
        adapted = proof
        
        for placeholder, orig_name in original_vars.items():
            if placeholder in new_vars:
                new_name = new_vars[placeholder]
                # Remplacer le nom original par le nouveau
                adapted = re.sub(rf'\b{orig_name}\b', new_name, adapted)
        
        return adapted
    
    def get_statistics(self) -> Dict:
        """Retourne des statistiques sur la memoire."""
        return {
            "total_patterns": len(self.proofs),
            "total_uses": sum(p.success_count for p in self.proofs.values()),
            "most_used": max(self.proofs.values(), 
                           key=lambda p: p.success_count).theorem_pattern 
                          if self.proofs else None
        }
    
    def save(self, filepath: str):
        """Sauvegarde la memoire dans un fichier JSON."""
        data = {
            pattern: {
                "theorem_pattern": sp.theorem_pattern,
                "original_theorem": sp.original_theorem,
                "proof": sp.proof,
                "success_count": sp.success_count,
                "variables": sp.variables
            }
            for pattern, sp in self.proofs.items()
        }
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=2)
    
    def load(self, filepath: str):
        """Charge la memoire depuis un fichier JSON."""
        with open(filepath, 'r') as f:
            data = json.load(f)
        
        self.proofs = {
            pattern: StoredProof(**stored)
            for pattern, stored in data.items()
        }

# Test de ProofMemory
print("Test de ProofMemory:")
print("-" * 50)

memory = ProofMemory()

# Stocker quelques preuves
memory.store(
    "theorem add_zero_n (n : Nat) : n + 0 = n",
    "exact Nat.add_zero n"
)
memory.store(
    "theorem add_comm_ab (a b : Nat) : a + b = b + a",
    "exact Nat.add_comm a b"
)

print(f"Preuves stockees: {len(memory.proofs)}")

# Tester le recall sur un theoreme similaire
test_theorem = "theorem my_add_zero (m : Nat) : m + 0 = m"
result = memory.recall(test_theorem)

if result:
    proof, score = result
    print(f"\nRecall pour '{test_theorem}':")
    print(f"  Score de similarite: {score:.2f}")
    print(f"  Preuve adaptee: {proof}")
else:
    print(f"\nPas de preuve trouvee pour '{test_theorem}'")

# Statistiques
stats = memory.get_statistics()
print(f"\nStatistiques memoire:")
print(f"  Patterns: {stats['total_patterns']}")
print(f"  Utilisations totales: {stats['total_uses']}")

Test de ProofMemory:
--------------------------------------------------
Preuves stockees: 2

Recall pour 'theorem my_add_zero (m : Nat) : m + 0 = m':
  Score de similarite: 1.00
  Preuve adaptee: exact Nat.add_zero m

Statistiques memoire:
  Patterns: 2
  Utilisations totales: 2


## Resume

### Architecture multi-agents pour theorem proving

| Agent | Role | Entrees | Sorties |
|-------|------|---------|--------|
| **OrchestratorAgent** | Coordonner workflow | Theoreme | Delegation + status |
| **SearchAgent** | Trouver lemmes Mathlib | But | Liste de lemmes |
| **TacticAgent** | Generer tactiques | But + lemmes | Sequence de tactiques |
| **VerifierAgent** | Valider avec Lean | Code Lean | Succes/Erreur + feedback |

### Patterns Semantic Kernel implementes

| Pattern | Description | Classe |
|---------|-------------|--------|
| **StateManager** | Etat partage entre agents | `ProofState` |
| **Plugin** | Fonctions @kernel_function | `LeanProverPlugin` |
| **SelectionStrategy** | Choix agent suivant | `DelegatingSelectionStrategy` |
| **TerminationStrategy** | Critere d'arret | `ProofCompleteTermination` |
| **AgentGroupChat** | Conversation multi-agents | `AgentGroupChat` |

### Techniques cles

1. **Etat partage** : Tous les agents lisent/ecrivent dans `ProofState`
2. **Delegation explicite** : Chaque agent designe le suivant via `delegate_to_agent`
3. **Boucle de feedback** : Echecs envoyes a `TacticAgent` pour correction
4. **Memoire de session** : Historique des tentatives pour eviter repetitions
5. **Decomposition (Aristotle)** : Diviser problemes complexes en sous-problemes

### Ressources et inspiration

| Source | Contribution |
|--------|--------------|
| **Argument_Analysis notebooks** | Patterns SK (StateManager, orchestration) |
| **Harmonic Aristotle** | Decomposition hierarchique, IMO Gold 2025 |
| **APOLLO** | Generation massive, filtrage par Lean |
| **AlphaProof** | RL + MCTS, Nature 2025 |
| **LeanDojo** | Extraction donnees, LeanCopilot |

### Impact futur

Les systemes agentiques pour theorem proving representent une nouvelle frontiere:
- **15+ problemes Erdos** resolus par IA depuis Noel 2025
- **Acceleration x10-100** de la formalisation mathematique
- **Decouverte** de nouvelles mathematiques par collaboration humain-IA
- **Verification formelle** comme standard de confiance absolue

---

*Notebook base sur les techniques de Harmonic Aristotle (IMO Gold 2025), APOLLO (arXiv 2505), AlphaProof (Nature 2025), et les patterns Semantic Kernel inspires de Argument_Analysis*

---

**Navigation** : [← Lean-7-LLM-Integration](Lean-7-LLM-Integration.ipynb) | [Index](Lean-1-Setup.ipynb) | [Lean-9-LeanDojo →](Lean-9-LeanDojo.ipynb)